In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

df = pd.read_stata('data/Authority.dta')
print(df.shape)
#list(df.columns)

#construct work category dummy
df['OG03_dummy'] = 0
df.loc[(df['work_category']=='OG03')&(df['work_category']!=''),'OG03_dummy'] = 1

df['OG01_dummy'] = 0
df.loc[(df['work_category']=='OG01')&(df['work_category']!=''),'OG01_dummy'] = 1

df['OG_rest_dummy'] = 0
df.loc[(df['OG01_dummy']!=1)&(df['OG03_dummy']!=1)&(df['work_category']!=''),'OG_rest_dummy'] = 1

df['OG_dummy'] = 0
df.loc[df['work_category'].str[0:2] == 'OG','OG_dummy'] = 1

df['OS_dummy'] = 0
df.loc[df['work_category'].str[0:2] == 'OS','OS_dummy'] = 1

#treated vs controls
df['trend'] = df['year'] - 1999

df['trend_treat'] = df['trend']
df.loc[(df['authority_code']!=3090272)&(df['authority_code']!=3070001),'trend_treat'] = 0
#15225 real change made / no zeros in df['trend']
#print(df['trend_treat'].value_counts()) chekced

df['trend_control'] = df['trend']
df.loc[(df['authority_code']==3090272)|(df['authority_code']==3070001),'trend_control'] = 0
#902 real change made
#print(df['trend_control'].value_counts()) checked

#PA specifics
df = df.sort_values(by='authority_code',ascending=True)
#auth = econ.group_id(df, cols = 'authority_code')
#print(auth) #dataframe

auth_list = df['authority_code'].values.tolist()
auth_list = list(set(auth_list))
#겹치는 부분 = authority_code야
#id_auth = group_id + 1 if df의 auth code == auth의 code

df['id_auth'] = 0
for i in range(len(df)):
    for j in range(len(auth_list)):
        if df.loc[i,'authority_code'] == auth_list[j]:
            df.loc[i,'id_auth'] = j+1

work_dum = pd.get_dummies(df['work_category'])
year_dum = pd.get_dummies(df['year'])
work_list = list(work_dum.columns)
year_list = list(year_dum.columns)

df_dum = pd.concat([year_dum, work_dum],axis = 1)
df = pd.concat([df, df_dum],axis = 1)

'''for i in range(len(auth_list)):
    name = 'trend_pa_'+str(i+1)
    df[name] = 0
    for j in range(len(df)):
        if df.loc[j, 'id_auth'] == i+1 :
            df.loc[j, name] = df.loc[j,'trend']
        if df.loc[j, 'authority_code']==3090272 or df.loc[j, 'authority_code']==3070001:
            df.loc[j, name] = 0'''
#for statement check            
#df.head
#print(max(df['id_path']))

(16127, 31)


"for i in range(len(auth_list)):\n    name = 'trend_pa_'+str(i+1)\n    df[name] = 0\n    for j in range(len(df)):\n        if df.loc[j, 'id_auth'] == i+1 :\n            df.loc[j, name] = df.loc[j,'trend']\n        if df.loc[j, 'authority_code']==3090272 or df.loc[j, 'authority_code']==3070001:\n            df.loc[j, name] = 0"

In [2]:
#!pip install linearmodels  
from linearmodels import PanelOLS
import statsmodels.api as sm

#2 econtools
#!pip install econtools
import econtools as econ
import econtools.metrics as mt

In [4]:
#Iwork 1~42 pattern unfound
#G인지 S인지 판단
#G이면 sorted된 거에서 인덱스 +1 =Iwork_nm
#S이면 sorted된 거에서 index + 17

''''work_ca = df['work_category'].values.tolist()
work_ca = list(set(work_ca))
#print(work_ca)
#total 42

os_list = []
og_list = []
for i in range(len(work_ca)):
    if work_ca[i][1]=='G':
        work_ca[i] = int(work_ca[i][2:])
        og_list.append(work_ca[i])
    else:
        work_ca[i] = int(work_ca[i][2:])
        os_list.append(work_ca[i])

os_list = sorted(os_list)
og_list = sorted(og_list)


print(df.loc[0,'work_category'][1])

work_ca_order = og_list + os_list
#work_ca_order 대신 (1,43)

for i in range(1,43):
    name = 'Iwork_ca_'+str(i+1) 
    df[name] = 0
    for j in range(len(df)):
        if df.loc[j,'work_category'][1] == 'G':
            work_OG = int(df.loc[j, 'work_category'][2:])
            for g in range(len(og_list)):
                if work_OG == og_list[g]:
                    idx_g = str(g+1)
                    df.loc[j, 'Iwork_ca_' + idx_g] = 1
        else:
            work_OS = int(df.loc[j, 'work_category'][2:])
            for s in range(len(os_list)):
                if work_OS == os_list[s]:
                    idx_s = str(s + 17)
                    df.loc[j, 'Iwork_ca_'+ idx_s] = 1'''

S


In [5]:
#2001~2010 dummy
'''for j in range(2001,2011):
    df['_Iyear_'+str(j)] = 0
    for i in range(len(df)):
        if df.loc[i,'year'] == j:
            df.loc[i, '_Iyear_'+str(j)] = 1'''

# Test  
* test용으로는 row1 col3: 'ctrl_exp_turin_co_sample'& discount만 쓸거야

In [85]:
#col2
#dm.ols(stats.api)
#sm.ols로 한번 나온 전적이 있음

df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
#if add
#year = pd.Categorical(df_reg_exp.year)
#id_auth = pd.Categorical(df_reg_exp.id_auth)
#df_reg_exp = df_reg_exp.set_index(['auth_anno','year'])

#df_reg_exp['year'] = year
#df_reg_exp['authority_code'] = id_auth
#exog_var = ['fpsb_auction','year','id_auth','reserve_price', 'municipality']

reg_col = []
for i in work_list:
    reg_col.append(i)
for j in year_list:
    reg_col.append(j)
exog_var = ['fpsb_auction','authority_code','reserve_price','municipality'] + reg_col 
x = df_reg_exp[exog_var]
y = df_reg_exp['discount']
#x = np.asarray(x)

model = sm.OLS(y,x).fit(cov_type='cluster',cov_kwds = {'groups': df_reg_exp['auth_anno']})
result = model.predict(x)
model.summary()
#value is different

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               discount   R-squared:                       0.590
Model:                            OLS   Adj. R-squared:                  0.577
Method:                 Least Squares   F-statistic:                       nan
Date:                Thu, 02 Jul 2020   Prob (F-statistic):                nan
Time:                        00:09:15   Log-Likelihood:                -3834.1
No. Observations:                1262   AIC:                             7748.
Df Residuals:                    1222   BIC:                             7954.
Df Model:                          39                                         
Covariance Type:              cluster                                         
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
fpsb_auction      11.5732      1.396      8.292      0.000       8.838      14.309
authority_code  2.086e-06   3.87e-07      5.394      0.000    1.33e-06    2.84e-06
reserve_price   1.678e-06   3.12e-07      5.377      0.000    1.07e-06    2.29e-06
municipality      -0.8983      0.626     -1.434      0.152      -2.126       0.330
OG01              -0.8561      0.532     -1.609      0.108      -1.899       0.187
OG02              -0.8443      0.575     -1.469      0.142      -1.971       0.282
OG03              -1.7278      0.537     -3.217      0.001      -2.780      -0.675
OG04              -5.9936      2.902     -2.065      0.039     -11.682      -0.305
OG05           -4.057e-15   5.21e-15     -0.779      0.436   -1.43e-14    6.15e-15
OG06               0.1156      0.595      0.194      0.846      -1.050       1.281
OG07               3.0838      5.278      0.584      0.559      -7.261      13.428
OG08               0.4782      1.582      0.302      0.762      -2.622       3.578
OG09            9.709e-15   1.38e-14      0.703      0.482   -1.74e-14    3.68e-14
OG10               9.1968      0.816     11.274      0.000       7.598      10.796
OG11               3.6730      0.897      4.097      0.000       1.916       5.430
OG12               6.1585      2.155      2.858      0.004       1.935      10.382
OG13            1.207e-14    4.9e-15      2.461      0.014    2.46e-15    2.17e-14
OG18            4.169e-15   1.77e-15      2.356      0.018       7e-16    7.64e-15
OG21            3.075e-15   1.88e-15      1.638      0.101   -6.04e-16    6.75e-15
OG24           -4.074e-15   1.86e-15     -2.188      0.029   -7.72e-15   -4.25e-16
OS01              -5.0225      2.740     -1.833      0.067     -10.392       0.347
OS02               6.4712      0.925      6.999      0.000       4.659       8.283
OS03           -4.399e-15   1.31e-15     -3.345      0.001   -6.98e-15   -1.82e-15
OS04            1.033e-15   1.68e-15      0.616      0.538   -2.25e-15    4.32e-15
OS05               7.8036      1.056      7.391      0.000       5.734       9.873
OS06               3.1012      1.504      2.062      0.039       0.154       6.048
OS07               1.9581      0.628      3.120      0.002       0.728       3.188
OS08               2.8030      3.037      0.923      0.356      -3.149       8.755
OS09           -4.716e-15    1.3e-15     -3.637      0.000   -7.26e-15   -2.17e-15
OS10              13.0574      1.602      8.150      0.000       9.917      16.198
OS11             -10.2344      1.018    -10.055      0.000     -12.229      -8.239
OS12              16.2861      1.235     13.189      0.000      13.866      18.706
OS13           -7.594e-15   8.62e-16     -8.814      0.000   -9.28e-15   -5.91e-15
OS14               2.4518      0.893      2.744      0.006       0.701       4.203
OS18        

In [50]:
#col2 0702 co&ctrl_exp&discount table2(2,1)
#sm.ols version2
#sm.regression
#nan with other outcomes except 'discount'

df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
#if add
#year = pd.Categorical(df_reg_exp.year)
#id_auth = pd.Categorical(df_reg_exp.id_auth)
#df_reg_exp = df_reg_exp.set_index(['auth_anno','year'])

#df_reg_exp['year'] = year
#df_reg_exp['authority_code'] = id_auth
#exog_var = ['fpsb_auction','year','id_auth','reserve_price', 'municipality']

reg_col = []
for i in work_list:
    reg_col.append(i)
for j in year_list:
    reg_col.append(j)
exog_var = ['fpsb_auction','authority_code','reserve_price','municipality'] + reg_col 
x = df_reg_exp[exog_var]
y = df_reg_exp['delay_ratio']
#exog = sm.add_constant(x)
mod = sm.OLS(y,x)
fe_reg = mod.fit(cov_type='cluster',cov_kwds = {'groups': df_reg_exp['auth_anno']})

#result = mod.predict(exog)
print(fe_reg.summary())

#multicollinearity

                            OLS Regression Results                            
Dep. Variable:            delay_ratio   R-squared:                         nan
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Thu, 02 Jul 2020   Prob (F-statistic):                nan
Time:                        11:39:24   Log-Likelihood:                    nan
No. Observations:                1262   AIC:                               nan
Df Residuals:                    1222   BIC:                               nan
Df Model:                          39                                         
Covariance Type:              cluster                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
fpsb_auction          nan        nan        

C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\stats.py:1548: RuntimeWarning: invalid value encountered in sign
  term2 = np.sign(denom) * np.where(denom == 0.0, np.nan,


In [19]:
# mutlicollinearity
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

In [25]:
#check multicollinearity
#will be problematic if VIF >=10
#exog_var = exog_var = ['fpsb_auction','id_auth','reserve_price','municipality'] + reg_col 
X = df_reg_exp.loc[:,exog_var]
vif = calc_vif(X)

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


In [83]:
#check multicollinearity
#will be problematic if VIF >=10
#exog_var = exog_var = ['fpsb_auction','authority_code','reserve_price','municipality'] + reg_col
#authority code로 넣어야 authority and municipality 의 mutilcolinearity없어짐
#아냐, multicolinearity는 없어져야 했음 id_auth를 넣고 multicolinearity 때문에 municipality를 삭제하면 됨

reg_col = []
for i in work_list:
    reg_col.append(i)
for j in year_list:
    reg_col.append(j)
exog_var = ['overrun_ratio','fpsb_auction','id_auth','reserve_price'] + reg_col 


X = df_reg_exp.loc[:,exog_var]
vif = calc_vif(X)
print(vif)

MissingDataError: exog contains inf or nans

In [69]:
#col2
#다중공선성 or NaN 미리 제거
for i in range(len(vif)):
    if np.isnan(vif.loc[i, 'VIF']) == True:
        col_name = vif.loc[i, 'variables']
        print(col_name)
        reg_col.remove(col_name)

print(reg_col)

OG05
OG09
OG13
OG18
OG21
OG24
OS03
OS04
OS09
OS13
OS32
OS33
2007.0
2008.0
2009.0
2010.0
['OG01', 'OG02', 'OG03', 'OG04', 'OG06', 'OG07', 'OG08', 'OG10', 'OG11', 'OG12', 'OS01', 'OS02', 'OS05', 'OS06', 'OS07', 'OS08', 'OS10', 'OS11', 'OS12', 'OS14', 'OS18', 'OS19', 'OS21', 'OS22', 'OS23', 'OS24', 'OS26', 'OS28', 'OS30', 'OS34', 2000.0, 2001.0, 2002.0, 2003.0, 2004.0, 2005.0, 2006.0]


In [70]:
#col2 0702 co&ctrl_exp&discount table2(2,1)
#sm.ols version3
#sm.regression
#prbm: nan with other outcomes except 'discount'
#trial : multicollinearity
#includes authority_code and deleted nan dummies

df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
#if add
#year = pd.Categorical(df_reg_exp.year)
#id_auth = pd.Categorical(df_reg_exp.id_auth)
#df_reg_exp = df_reg_exp.set_index(['auth_anno','year'])

#df_reg_exp['year'] = year
#df_reg_exp['authority_code'] = id_auth
#exog_var = ['fpsb_auction','year','id_auth','reserve_price', 'municipality']

#reg_col = []
#for i in work_list:
#    reg_col.append(i)
#for j in year_list:
#    reg_col.append(j)
exog_var = ['fpsb_auction','id_auth','reserve_price'] + reg_col 
x = df_reg_exp[exog_var]
y = df_reg_exp['delay_ratio']
#exog = sm.add_constant(x)
mod = sm.OLS(y,x, hasconst = True)
fe_reg = mod.fit(cov_type='cluster',cov_kwds = {'groups': df_reg_exp['auth_anno']})

#result = mod.predict(exog)
print(fe_reg.summary())

#multicollinearity

                            OLS Regression Results                            
Dep. Variable:            delay_ratio   R-squared:                         nan
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Thu, 02 Jul 2020   Prob (F-statistic):                nan
Time:                        11:48:47   Log-Likelihood:                    nan
No. Observations:                1262   AIC:                               nan
Df Residuals:                    1223   BIC:                               nan
Df Model:                          38                                         
Covariance Type:              cluster                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
fpsb_auction         nan        nan        nan

C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\stats.py:1548: RuntimeWarning: invalid value encountered in sign
  term2 = np.sign(denom) * np.where(denom == 0.0, np.nan,


In [71]:
#col2
#Panel OLS
#discount
#doesnot have full rank: set index 잘 만져서 해결했었음


df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
id_auth = pd.Categorical(df_reg_exp.id_auth)
#work_category = pd.Categorical(df_reg_exp.work_category)
year = pd.Categorical(df_reg_exp.year)

df_reg_exp = df_reg_exp.set_index(['auth_anno','year'])
df_reg_exp['year'] = year
df_reg_exp['authority_code'] = id_auth
#df_reg_exp['work_category'] = work_category


#Iwork_category = []
#for i in range(len(work_ca_order)):
#    Iwork_category.append('Iwork_ca_'+str(i+1))
#reg_col = []
#for i in work_list:
#    reg_col.append(i)
#for j in year_list:
#    reg_col.append(j)
exog_var = ['fpsb_auction','id_auth','year','reserve_price'] + reg_col 
#exog_var = np.array(exog_var)
exog = sm.add_constant(df_reg_exp[exog_var])
mod = PanelOLS(df_reg_exp.discount, exog, time_effects=True, drop_absorbed = True)  
fe_res = mod.fit()
print(fe_res)

#exog does not have full column rank 문제에서 계속 막힘

ValueError: exog does not have full column rank.

In [72]:
#mt.reg
#singular matrix error 너무 걸려서 못 씀
import econtools as econ
import econtools.metrics as mt

df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
mt.reg(df_reg_exp,'delay_ratio',['fpsb_auction','reserve_price'] + reg_col, fe_name = 'authority_code', cluster = 'auth_anno')

C:\Users\thxkn\anaconda3\lib\site-packages\econtools\metrics\core.py:203: RuntimeWarning: invalid value encountered in sqrt
  se = pd.Series(np.sqrt(np.diagonal(vce)), index=vce.columns)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1827: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= np.asarray(_b)) & cond0


Dependent variable:	delay_ratio
N:			1110
R-squared:		0.1216
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  99
Fixed effects by:	authority_code
  No. of FE:		  15
                             coeff                  se      t   p>t                CI_low             CI_high
fpsb_auction                35.300              11.959  2.952 0.004                11.569              59.032
reserve_price               -0.000               0.000 -2.008 0.047                -0.000              -0.000
OG01          -580478055792879.000 587425575418275.250 -0.988 0.326 -1746204992108318.500 585248880522560.500
OG02          -580478055792879.000 587425575418268.500 -0.988 0.326 -1746204992108305.000 585248880522547.000
OG03          -580478055792872.000 587425575418274.375 -0.988 0.326 -1746204992108309.750 585248880522565.750
OG04          -580478055792900.000 587425575418266.000 -0.988 0.326 -1746204992108321.250 585248880522521.250
OG06          -58

# Col2: value는 달라도 그냥 고/ sm.OLS

In [93]:
#indexing 하는 순간 에러남
#outcome = ['discount', 'delay_ratio', 'overrun_ratio', 'days_to_award']

df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
reg_col = []
for i in work_list:
    reg_col.append(i)
for j in year_list:
    reg_col.append(j)
exog_var = ['fpsb_auction','authority_code','reserve_price','municipality'] + reg_col 
x = df_reg_exp[exog_var]

y = df_reg_exp['discount']
model = sm.OLS(y,x).fit(cov_type='cluster',cov_kwds = {'groups': df_reg_exp['auth_anno']})
result = model.predict(x)
model.summary()
#value diff
#문제점 1: omitted되어야 할 게 omitted되지 않음
#문제점 2: outcome 바꾸면 nan값으로만 나옴: r_matrix performs f_test for using dimensions that are asymptotically non-normal

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               discount   R-squared:                       0.590
Model:                            OLS   Adj. R-squared:                  0.577
Method:                 Least Squares   F-statistic:                       nan
Date:                Thu, 02 Jul 2020   Prob (F-statistic):                nan
Time:                        00:16:29   Log-Likelihood:                -3834.1
No. Observations:                1262   AIC:                             7748.
Df Residuals:                    1222   BIC:                             7954.
Df Model:                          39                                         
Covariance Type:              cluster                                         
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
fpsb_auction      11.5732      1.396      8.292      0.000       8.838      14.309
authority_code  2.086e-06   3.87e-07      5.394      0.000    1.33e-06    2.84e-06
reserve_price   1.678e-06   3.12e-07      5.377      0.000    1.07e-06    2.29e-06
municipality      -0.8983      0.626     -1.434      0.152      -2.126       0.330
OG01              -0.8561      0.532     -1.609      0.108      -1.899       0.187
OG02              -0.8443      0.575     -1.469      0.142      -1.971       0.282
OG03              -1.7278      0.537     -3.217      0.001      -2.780      -0.675
OG04              -5.9936      2.902     -2.065      0.039     -11.682      -0.305
OG05           -4.057e-15   5.21e-15     -0.779      0.436   -1.43e-14    6.15e-15
OG06               0.1156      0.595      0.194      0.846      -1.050       1.281
OG07               3.0838      5.278      0.584      0.559      -7.261      13.428
OG08               0.4782      1.582      0.302      0.762      -2.622       3.578
OG09            9.709e-15   1.38e-14      0.703      0.482   -1.74e-14    3.68e-14
OG10               9.1968      0.816     11.274      0.000       7.598      10.796
OG11               3.6730      0.897      4.097      0.000       1.916       5.430
OG12               6.1585      2.155      2.858      0.004       1.935      10.382
OG13            1.207e-14    4.9e-15      2.461      0.014    2.46e-15    2.17e-14
OG18            4.169e-15   1.77e-15      2.356      0.018       7e-16    7.64e-15
OG21            3.075e-15   1.88e-15      1.638      0.101   -6.04e-16    6.75e-15
OG24           -4.074e-15   1.86e-15     -2.188      0.029   -7.72e-15   -4.25e-16
OS01              -5.0225      2.740     -1.833      0.067     -10.392       0.347
OS02               6.4712      0.925      6.999      0.000       4.659       8.283
OS03           -4.399e-15   1.31e-15     -3.345      0.001   -6.98e-15   -1.82e-15
OS04            1.033e-15   1.68e-15      0.616      0.538   -2.25e-15    4.32e-15
OS05               7.8036      1.056      7.391      0.000       5.734       9.873
OS06               3.1012      1.504      2.062      0.039       0.154       6.048
OS07               1.9581      0.628      3.120      0.002       0.728       3.188
OS08               2.8030      3.037      0.923      0.356      -3.149       8.755
OS09           -4.716e-15    1.3e-15     -3.637      0.000   -7.26e-15   -2.17e-15
OS10              13.0574      1.602      8.150      0.000       9.917      16.198
OS11             -10.2344      1.018    -10.055      0.000     -12.229      -8.239
OS12              16.2861      1.235     13.189      0.000      13.866      18.706
OS13           -7.594e-15   8.62e-16     -8.814      0.000   -9.28e-15   -5.91e-15
OS14               2.4518      0.893      2.744      0.006       0.701       4.203
OS18        

In [ ]:
#vif한 후에 돌리기

y = df_reg_exp['discount']
model = sm.OLS(y,x,missing = 'drop').fit(cov_type='cluster',cov_kwds = {'groups': df_reg_exp['auth_anno']})
#result = model.predict(x)
print(model.summary())

In [76]:
import econtools as econ
import econtools.metrics as mt

df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]

# Cal2 = multicolinearity check and then delete NaN or vif > 10

In [10]:
#col2 setting
df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]

In [57]:
# mutlicollinearity function
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

#vif cal
#first, make a column list
reg_col = []
for i in work_list:
    reg_col.append(i)
for j in year_list:
    reg_col.append(j)
exog_var = ['fpsb_auction','id_auth','reserve_price','municipality']
exog = exog_var + reg_col 


#check multicollinearity
X = df_reg_exp.loc[:,exog]
vif = calc_vif(X)
print(vif)


#delete from col list
for i in range(len(vif)):
    if np.isnan(vif.loc[i, 'VIF']) == True:
        reg_col.remove(vif.loc[i, 'variables'])
    elif vif.loc[i,'VIF'] > 10:
        for j in exog_var:
            if str(vif.loc[i,'variables']) is j and vif.loc[i,'variables'] is not 'fpsb_auction' and vif.loc[i,'variables'] is not 'id_auth':
                exog_var.remove(vif.loc[i,'variables'])
                
exog = exog_var + reg_col
exog.remove('id_auth')
exog.remove(2000)
exog.remove('OG01')

print(exog) #check


C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


        variables        VIF
0    fpsb_auction   1.508187
1         id_auth  36.312618
2   reserve_price   1.129392
3    municipality  34.669915
4            OG01        inf
5            OG02        inf
6            OG03        inf
7            OG04        inf
8            OG05        NaN
9            OG06        inf
10           OG07        inf
11           OG08        inf
12           OG09        NaN
13           OG10        inf
14           OG11        inf
15           OG12        inf
16           OG13        NaN
17           OG18        NaN
18           OG21        NaN
19           OG24        NaN
20           OS01        inf
21           OS02        inf
22           OS03        NaN
23           OS04        NaN
24           OS05        inf
25           OS06        inf
26           OS07        inf
27           OS08        inf
28           OS09        NaN
29           OS10        inf
30           OS11        inf
31           OS12        inf
32           OS13        NaN
33           O

In [40]:
#winning discount
mt.reg(df_reg_exp,'discount', exog, fe_name = 'authority_code', cluster = 'auth_anno')
#exog에서 id_auth 빼는 거 잊지 말 것

Dependent variable:	discount
N:			1262
R-squared:		0.6393
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  101
Fixed effects by:	authority_code
  No. of FE:		  15
               coeff    se      t   p>t  CI_low CI_high
fpsb_auction  11.992 1.315  9.117 0.000   9.382  14.602
reserve_price  0.000 0.000  5.993 0.000   0.000   0.000
OG02           0.205 0.629  0.326 0.745  -1.043   1.452
OG03          -0.583 0.545 -1.070 0.287  -1.665   0.498
OG04          -4.754 3.344 -1.422 0.158 -11.389   1.881
OG06           0.852 0.655  1.300 0.196  -0.448   2.151
OG07           4.509 5.121  0.881 0.381  -5.651  14.668
OG08           0.801 1.606  0.499 0.619  -2.384   3.986
OG10           8.419 0.515 16.337 0.000   7.396   9.441
OG11           5.680 1.058  5.367 0.000   3.580   7.780
OG12           6.017 1.940  3.101 0.003   2.168   9.866
OS01          -2.534 1.204 -2.104 0.038  -4.923  -0.145
OS02           9.521 1.405  6.777 0.000   6.734  12.308
OS05 

In [41]:
#delay_ratio
mt.reg(df_reg_exp,'delay_ratio', exog, fe_name = 'authority_code', cluster = 'auth_anno')

Dependent variable:	delay_ratio
N:			1110
R-squared:		0.1221
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  99
Fixed effects by:	authority_code
  No. of FE:		  15
                coeff     se      t   p>t  CI_low CI_high
fpsb_auction   34.180 12.127  2.818 0.006  10.113  58.246
reserve_price  -0.000  0.000 -2.001 0.048  -0.000  -0.000
OG02           -0.694 10.337 -0.067 0.947 -21.207  19.819
OG03            8.270  5.345  1.547 0.125  -2.338  18.878
OG04          -16.063  6.684 -2.403 0.018 -29.327  -2.799
OG06           31.080 10.965  2.834 0.006   9.320  52.840
OG07           45.994 24.442  1.882 0.063  -2.510  94.498
OG08           14.479 14.907  0.971 0.334 -15.104  44.061
OG10           57.231 11.041  5.184 0.000  35.321  79.141
OG11            4.508 12.597  0.358 0.721 -20.491  29.507
OG12           40.030 50.648  0.790 0.431 -60.479 140.539
OS01           -5.251  6.614 -0.794 0.429 -18.376   7.875
OS02           11.113  8.158  1.3

In [73]:
#overrun_ratio (problem)
#nan overrun ratio
#안먹히면 OLS nan 값 제거하고 넣기?
#먼저 overrun_ratio drop
#col2 setting

mt.reg(df_reg_exp,'overrun_ratio', exog, fe_name = 'authority_code', cluster = 'auth_anno')

LinAlgError: Singular matrix

In [74]:
#days_to_award
mt.reg(df_reg_exp,'days_to_award', exog, fe_name = 'authority_code', cluster = 'auth_anno')

LinAlgError: Singular matrix

# Col 4: ctrl_pop& iwork

In [76]:
#col4 setting
df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_pop_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]

In [79]:
# mutlicollinearity function
#from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

#vif cal
#first, make a column list
reg_col = []
for i in work_list:
    reg_col.append(i)
for j in year_list:
    reg_col.append(j)
exog_var = ['fpsb_auction','id_auth','reserve_price','municipality']
exog = exog_var + reg_col 


#check multicollinearity
X = df_reg_exp.loc[:,exog]
vif = calc_vif(X)
print(vif)


#delete from col list
for i in range(len(vif)):
    if np.isnan(vif.loc[i, 'VIF']) == True:
        reg_col.remove(vif.loc[i, 'variables'])
    '''elif vif.loc[i,'VIF'] > 10:
           for j in exog_var:
            if str(vif.loc[i,'variables']) is j and vif.loc[i,'variables'] is not 'fpsb_auction' and vif.loc[i,'variables'] is not 'id_auth':
                exog_var.remove(vif.loc[i,'variables'])'''
#mun기냥 직접 지워
                
exog = exog_var + reg_col

exog.remove('municipality')
exog.remove('id_auth')
exog.remove(2000)
exog.remove('OG01')

print(exog) #check


C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


        variables       VIF
0    fpsb_auction  1.574905
1         id_auth  9.804265
2   reserve_price  1.105848
3    municipality  8.944864
4            OG01       inf
5            OG02       inf
6            OG03       inf
7            OG04       inf
8            OG05       NaN
9            OG06       inf
10           OG07       inf
11           OG08       inf
12           OG09       inf
13           OG10       inf
14           OG11       inf
15           OG12       inf
16           OG13       inf
17           OG18       NaN
18           OG21       NaN
19           OG24       NaN
20           OS01       inf
21           OS02       inf
22           OS03       NaN
23           OS04       NaN
24           OS05       inf
25           OS06       inf
26           OS07       inf
27           OS08       inf
28           OS09       NaN
29           OS10       inf
30           OS11       inf
31           OS12       inf
32           OS13       inf
33           OS14       NaN
34           OS18   

In [80]:
#(1,4)winning discount
mt.reg(df_reg_exp,'discount', exog, fe_name = 'authority_code', cluster = 'auth_anno')
#exog에서 id_auth 빼는 거 잊지 말 것

#delay_ratio
#mt.reg(df_reg_exp,'delay_ratio', exog, fe_name = 'authority_code', cluster = 'auth_anno')

Dependent variable:	discount
N:			1275
R-squared:		0.6140
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  145
Fixed effects by:	authority_code
  No. of FE:		  24
               coeff    se      t   p>t CI_low CI_high
fpsb_auction  12.023 1.469  8.187 0.000  9.120  14.926
reserve_price  0.000 0.000  5.221 0.000  0.000   0.000
OG02          -0.045 0.694 -0.065 0.948 -1.416   1.326
OG03          -0.696 0.582 -1.197 0.233 -1.845   0.454
OG04          -0.326 0.557 -0.585 0.560 -1.426   0.775
OG06          -3.037 1.465 -2.073 0.040 -5.933  -0.141
OG07           0.948 2.510  0.378 0.706 -4.014   5.910
OG08           1.041 1.632  0.638 0.525 -2.186   4.267
OG09           2.600 1.163  2.236 0.027  0.301   4.898
OG10           8.371 0.551 15.183 0.000  7.282   9.461
OG11           5.921 1.061  5.582 0.000  3.824   8.017
OG12           6.018 2.011  2.993 0.003  2.043   9.992
OG13          -2.649 1.176 -2.252 0.026 -4.973  -0.324
OS01          -1.32

In [82]:
#(3,4)completion time delay
mt.reg(df_reg_exp,'delay_ratio', exog, fe_name = 'authority_code', cluster = 'auth_anno')

LinAlgError: Singular matrix

In [83]:
mt.reg(df_reg_exp,'overrun_ratio', exog, fe_name = 'authority_code', cluster = 'auth_anno')

LinAlgError: Singular matrix

In [84]:
mt.reg(df_reg_exp,'days_to_award', exog, fe_name = 'authority_code', cluster = 'auth_anno')

LinAlgError: Singular matrix

# Col 6: ctrl_pop_exp

In [87]:
#col4 setting
df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_pop_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]

# mutlicollinearity function
#from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

#vif cal
#first, make a column list
reg_col = []
for i in work_list:
    reg_col.append(i)
for j in year_list:
    reg_col.append(j)
exog_var = ['fpsb_auction','id_auth','reserve_price','municipality']
exog = exog_var + reg_col 


#check multicollinearity
X = df_reg_exp.loc[:,exog]
vif = calc_vif(X)
print(vif)


#delete from col list
for i in range(len(vif)):
    if np.isnan(vif.loc[i, 'VIF']) == True:
        reg_col.remove(vif.loc[i, 'variables'])
    '''elif vif.loc[i,'VIF'] > 10:
           for j in exog_var:
            if str(vif.loc[i,'variables']) is j and vif.loc[i,'variables'] is not 'fpsb_auction' and vif.loc[i,'variables'] is not 'id_auth':
                exog_var.remove(vif.loc[i,'variables'])'''
#mun기냥 직접 지워
                
exog = exog_var + reg_col

exog.remove('municipality')
exog.remove('id_auth')
exog.remove(2000)
exog.remove('OG01')

print(exog) #check


C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


        variables        VIF
0    fpsb_auction   1.844743
1         id_auth  32.812465
2   reserve_price   1.204136
3    municipality  29.847522
4            OG01        inf
5            OG02        inf
6            OG03        inf
7            OG04        inf
8            OG05        NaN
9            OG06        inf
10           OG07        inf
11           OG08        inf
12           OG09        NaN
13           OG10        inf
14           OG11        inf
15           OG12        inf
16           OG13        NaN
17           OG18        NaN
18           OG21        NaN
19           OG24        NaN
20           OS01        inf
21           OS02        inf
22           OS03        NaN
23           OS04        NaN
24           OS05        inf
25           OS06        inf
26           OS07        inf
27           OS08        inf
28           OS09        NaN
29           OS10        inf
30           OS11        inf
31           OS12        inf
32           OS13        NaN
33           O

In [88]:
#(1,6)winning discount
mt.reg(df_reg_exp,'discount', exog, fe_name = 'authority_code', cluster = 'auth_anno')

Dependent variable:	discount
N:			880
R-squared:		0.6444
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  73
Fixed effects by:	authority_code
  No. of FE:		  11
               coeff    se      t   p>t CI_low CI_high
fpsb_auction  12.258 1.380  8.880 0.000  9.506  15.009
reserve_price  0.000 0.000  4.255 0.000  0.000   0.000
OG02          -0.958 0.791 -1.212 0.230 -2.535   0.618
OG03          -0.561 0.745 -0.753 0.454 -2.047   0.924
OG04          -0.382 0.725 -0.527 0.600 -1.828   1.063
OG06          -1.935 1.355 -1.429 0.157 -4.636   0.765
OG07           4.209 5.161  0.816 0.417 -6.079  14.497
OG08           0.524 1.660  0.316 0.753 -2.786   3.834
OG10           8.428 0.618 13.632 0.000  7.195   9.660
OG11           5.903 1.475  4.003 0.000  2.963   8.843
OG12           6.723 2.211  3.040 0.003  2.315  11.131
OS01          -2.257 1.326 -1.702 0.093 -4.899   0.386
OS02           9.543 1.465  6.512 0.000  6.622  12.464
OS05           5.700 

In [89]:
#(3,6)completion
mt.reg(df_reg_exp,'delay_ratio', exog, fe_name = 'authority_code', cluster = 'auth_anno')

Dependent variable:	delay_ratio
N:			747
R-squared:		0.1472
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  71
Fixed effects by:	authority_code
  No. of FE:		  11
                coeff     se      t   p>t  CI_low CI_high
fpsb_auction   39.285 12.001  3.274 0.002  15.350  63.219
reserve_price  -0.000  0.000 -2.248 0.028  -0.000  -0.000
OG02           -7.784 11.446 -0.680 0.499 -30.612  15.044
OG03            4.890  7.126  0.686 0.495  -9.324  19.103
OG04          -16.448 10.805 -1.522 0.132 -37.998   5.101
OG06          109.160 19.826  5.506 0.000  69.618 148.701
OG07           47.214 26.145  1.806 0.075  -4.931  99.359
OG08            8.643 15.792  0.547 0.586 -22.854  40.140
OG10           54.134 12.030  4.500 0.000  30.140  78.128
OG11            8.686 14.587  0.595 0.553 -20.406  37.778
OG12           38.608 63.160  0.611 0.543 -87.360 164.577
OS01           -7.694  7.427 -1.036 0.304 -22.508   7.119
OS02            7.514  9.203  0.81

In [90]:
#(2,6)cost overrun
mt.reg(df_reg_exp,'overrun_ratio', exog, fe_name = 'authority_code', cluster = 'auth_anno')

LinAlgError: Singular matrix

In [91]:
#(4,6)awarding
mt.reg(df_reg_exp,'days_to_award', exog, fe_name = 'authority_code', cluster = 'auth_anno')

LinAlgError: Singular matrix

In [136]:
#(4,6) days to award/ pop_exp OLS로 도전
#authority_code dummy로 만들기 absorb가 없으니깐
auth_dum = pd.get_dummies(df['id_auth'])
auth_list = list(auth_dum.columns)
dft = pd.concat([df,auth_dum],axis = 1)

df_reg_exp = dft[(dft['turin_co_sample']==1)&(dft['ctrl_pop_exp_turin_co_sample']==1)&(dft['post_experience']>= 5) & (dft['pre_experience']>=5) &(dft['post_experience'].isnull() == False ) & (dft['pre_experience'].isnull()==False)&(dft['missing']==0)]
#df_reg_exp.sort_values(by='id_auth',ascending= True)
dft.head()
df_reg_exp.head()
print(df_reg_exp.shape)

(880, 2092)


In [149]:
#auth_dum 추가해서 multicollinearity check
# mutlicollinearity function
#from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

#vif cal
#first, make a column list
reg_col = []
for i in work_list:
    reg_col.append(i)
for j in year_list:
    reg_col.append(j)
for k in auth_list:
    reg_col.append(k)

exog_var = ['fpsb_auction','reserve_price','municipality'] #id_auth삭제 왜냐면 id_auth_dum으로 살펴볼거니깐
exog = exog_var + reg_col 

#check multicollinearity
#시간 너무 오래걸림
X = df_reg_exp.loc[:,exog]
vif = calc_vif(X)
print(vif)


#delete from col list
for i in range(len(vif)):
    if np.isnan(vif.loc[i, 'VIF']) == True:
        reg_col.remove(vif.loc[i, 'variables'])
#    '''elif vif.loc[i,'VIF'] > 10:
#           for j in exog_var:
#            if str(vif.loc[i,'variables']) is j and vif.loc[i,'variables'] is not 'fpsb_auction' and vif.loc[i,'variables'] is not 'id_auth':
#                exog_var.remove(vif.loc[i,'variables'])'''
#mun기냥 직접 지워 단 multicollinearity check후 지워야 함 아니면 다중공선성이 달라질 수도 있으니깐
                
exog = exog_var + reg_col

exog.remove('municipality')
exog.remove('id_auth')
exog.remove(2000)
exog.remove('OG01')

KeyboardInterrupt: 

In [ ]:
#reg_col = vif에 있는 거
#exog_var = ['fpsb_auction','authority_code','reserve_price'] + reg_col 
#iauthority 엄청 삭제됨
x = df_reg_exp[exog]
y = df_reg_exp['days_to_award']
model = sm.OLS(y,x).fit(cov_type='cluster',cov_kwds = {'groups': df_reg_exp['auth_anno']})
result = model.predict(x)
#model.summary()

In [148]:
authority = df['authority_code'].values.tolist()
authority = list(set(authority))
print(authority)

[2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 82.0, 83.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, 101.0, 102.0, 103.0, 104.0, 105.0, 106.0, 107.0, 108.0, 109.0, 110.0, 111.0, 112.0, 113.0, 114.0, 115.0, 116.0, 117.0, 118.0, 119.0, 120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126.0, 127.0, 128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134.0, 135.0, 136.0, 137.0, 138.0, 139.0, 140.0, 141.0, 142.0, 143.0, 144.0, 145.0, 146.0, 147.0, 148.0, 149.0, 150.0, 151.0, 152.0, 153.0, 154.0, 155.0, 156.0, 157.0, 158.0, 159.0, 160.0, 161.0, 162.0, 163.0, 1

note: _Iauthority_5 omitted because of collinearity
note: _Iauthority_7 omitted because of collinearity


note: _Iauthority_10 omitted because of collinearity
note: _Iauthority_11 omitted because of collinearity
note: _Iauthority_12 omitted because of collinearity
note: _Iauthority_13 omitted because of collinearity
note: _Iauthority_14 omitted because of collinearity
note: _Iauthority_15 omitted because of collinearity

note: _Iauthority_17 omitted because of collinearity
note: _Iauthority_18 omitted because of collinearity
note: _Iauthority_19 omitted because of collinearity

note: _Iauthority_21 omitted because of collinearity
note: _Iauthority_22 omitted because of collinearity

note: _Iauthority_33 omitted because of collinearity
note: _Iauthority_35 omitted because of collinearity
note: _Iauthority_36 omitted because of collinearity
note: _Iauthority_37 omitted because of collinearity
note: _Iauthority_39 omitted because of collinearity

note: _Iauthority_41 omitted because of collinearity
note: _Iauthority_42 omitted because of collinearity
note: _Iauthority_43 omitted because of collinearity
note: _Iauthority_44 omitted because of collinearity
note: _Iauthority_45 omitted because of collinearity

note: _Iauthority_46 omitted because of collinearity
note: _Iauthority_47 omitted because of collinearity
note: _Iauthority_48 omitted because of collinearity
note: _Iauthority_49 omitted because of collinearity

note: _Iauthority_51 omitted because of collinearity
note: _Iauthority_52 omitted because of collinearity
note: _Iauthority_53 omitted because of collinearity
note: _Iauthority_54 omitted because of collinearity
note: _Iauthority_55 omitted because of collinearity

note: _Iauthority_56 omitted because of collinearity
note: _Iauthority_57 omitted because of collinearity
note: _Iauthority_58 omitted because of collinearity
note: _Iauthority_59 omitted because of collinearity
note: _Iauthority_60 omitted because of collinearity

note: _Iauthority_61 omitted because of collinearity
note: _Iauthority_62 omitted because of collinearity
note: _Iauthority_63 omitted because of collinearity
note: _Iauthority_64 omitted because of collinearity
note: _Iauthority_65 omitted because of collinearity
note: _Iauthority_66 omitted because of collinearity
note: _Iauthority_67 omitted because of collinearity
note: _Iauthority_68 omitted because of collinearity
note: _Iauthority_69 omitted because of collinearity
note: _Iauthority_70 omitted because of collinearity

note: _Iauthority_71 omitted because of collinearity
note: _Iauthority_72 omitted because of collinearity
note: _Iauthority_73 omitted because of collinearity
note: _Iauthority_74 omitted because of collinearity
note: _Iauthority_75 omitted because of collinearity
note: _Iauthority_76 omitted because of collinearity
note: _Iauthority_77 omitted because of collinearity
note: _Iauthority_78 omitted because of collinearity
note: _Iauthority_79 omitted because of collinearity
note: _Iauthority_80 omitted because of collinearity

note: _Iauthority_82 omitted because of collinearity
note: _Iauthority_83 omitted because of collinearity
note: _Iauthority_85 omitted because of collinearity
note: _Iauthority_86 omitted because of collinearity
note: _Iauthority_87 omitted because of collinearity
note: _Iauthority_88 omitted because of collinearity
note: _Iauthority_89 omitted because of collinearity

note: _Iauthority_90 omitted because of collinearity
note: _Iauthority_92 omitted because of collinearity
note: _Iauthority_93 omitted because of collinearity
note: _Iauthority_94 omitted because of collinearity
note: _Iauthority_95 omitted because of collinearity
note: _Iauthority_96 omitted because of collinearity
note: _Iauthority_97 omitted because of collinearity
note: _Iauthority_98 omitted because of collinearity
note: _Iauthority_99 omitted because of collinearity
note: _Iauthority_100 omitted because of collinearity

note: _Iauthority_101 omitted because of collinearity
note: _Iauthority_102 omitted because of collinearity
note: _Iauthority_103 omitted because of collinearity
note: _Iauthority_104 omitted because of collinearity
note: _Iauthority_105 omitted because of collinearity
note: _Iauthority_106 omitted because of collinearity
note: _Iauthority_107 omitted because of collinearity
note: _Iauthority_108 omitted because of collinearity
note: _Iauthority_109 omitted because of collinearity
note: _Iauthority_110 omitted because of collinearity

note: _Iauthority_111 omitted because of collinearity
note: _Iauthority_112 omitted because of collinearity
note: _Iauthority_113 omitted because of collinearity
note: _Iauthority_114 omitted because of collinearity
note: _Iauthority_115 omitted because of collinearity
note: _Iauthority_116 omitted because of collinearity
note: _Iauthority_117 omitted because of collinearity
note: _Iauthority_118 omitted because of collinearity
note: _Iauthority_119 omitted because of collinearity
note: _Iauthority_120 omitted because of collinearity
note: _Iauthority_121 omitted because of collinearity
note: _Iauthority_122 omitted because of collinearity
note: _Iauthority_123 omitted because of collinearity
note: _Iauthority_124 omitted because of collinearity
note: _Iauthority_125 omitted because of collinearity
note: _Iauthority_126 omitted because of collinearity
note: _Iauthority_127 omitted because of collinearity
note: _Iauthority_128 omitted because of collinearity
note: _Iauthority_129 omitted because of collinearity
note: _Iauthority_130 omitted because of collinearity
note: _Iauthority_131 omitted because of collinearity
note: _Iauthority_132 omitted because of collinearity
note: _Iauthority_133 omitted because of collinearity
note: _Iauthority_134 omitted because of collinearity
note: _Iauthority_135 omitted because of collinearity
note: _Iauthority_136 omitted because of collinearity
note: _Iauthority_137 omitted because of collinearity
note: _Iauthority_138 omitted because of collinearity
note: _Iauthority_139 omitted because of collinearity
note: _Iauthority_140 omitted because of collinearity
note: _Iauthority_141 omitted because of collinearity
note: _Iauthority_142 omitted because of collinearity
note: _Iauthority_143 omitted because of collinearity
note: _Iauthority_144 omitted because of collinearity
note: _Iauthority_145 omitted because of collinearity
note: _Iauthority_146 omitted because of collinearity
note: _Iauthority_147 omitted because of collinearity
note: _Iauthority_148 omitted because of collinearity
note: _Iauthority_149 omitted because of collinearity
note: _Iauthority_150 omitted because of collinearity
note: _Iauthority_151 omitted because of collinearity
note: _Iauthority_152 omitted because of collinearity
note: _Iauthority_153 omitted because of collinearity
note: _Iauthority_154 omitted because of collinearity
note: _Iauthority_155 omitted because of collinearity
note: _Iauthority_156 omitted because of collinearity
note: _Iauthority_157 omitted because of collinearity
note: _Iauthority_158 omitted because of collinearity
note: _Iauthority_159 omitted because of collinearity
note: _Iauthority_160 omitted because of collinearity
note: _Iauthority_161 omitted because of collinearity
note: _Iauthority_162 omitted because of collinearity
note: _Iauthority_163 omitted because of collinearity
note: _Iauthority_164 omitted because of collinearity
note: _Iauthority_165 omitted because of collinearity
note: _Iauthority_166 omitted because of collinearity
note: _Iauthority_167 omitted because of collinearity
note: _Iauthority_168 omitted because of collinearity
note: _Iauthority_169 omitted because of collinearity
note: _Iauthority_170 omitted because of collinearity
note: _Iauthority_171 omitted because of collinearity
note: _Iauthority_172 omitted because of collinearity
note: _Iauthority_173 omitted because of collinearity
note: _Iauthority_174 omitted because of collinearity
note: _Iauthority_175 omitted because of collinearity
note: _Iauthority_176 omitted because of collinearity
note: _Iauthority_177 omitted because of collinearity
note: _Iauthority_178 omitted because of collinearity
note: _Iauthority_179 omitted because of collinearity
note: _Iauthority_180 omitted because of collinearity
note: _Iauthority_181 omitted because of collinearity
note: _Iauthority_182 omitted because of collinearity
note: _Iauthority_183 omitted because of collinearity
note: _Iauthority_184 omitted because of collinearity
note: _Iauthority_185 omitted because of collinearity
note: _Iauthority_186 omitted because of collinearity
note: _Iauthority_187 omitted because of collinearity
note: _Iauthority_188 omitted because of collinearity
note: _Iauthority_189 omitted because of collinearity
note: _Iauthority_190 omitted because of collinearity
note: _Iauthority_191 omitted because of collinearity
note: _Iauthority_192 omitted because of collinearity
note: _Iauthority_193 omitted because of collinearity
note: _Iauthority_194 omitted because of collinearity
note: _Iauthority_195 omitted because of collinearity
note: _Iauthority_196 omitted because of collinearity
note: _Iauthority_197 omitted because of collinearity
note: _Iauthority_198 omitted because of collinearity
note: _Iauthority_199 omitted because of collinearity
note: _Iauthority_200 omitted because of collinearity
note: _Iauthority_201 omitted because of collinearity
note: _Iauthority_202 omitted because of collinearity
note: _Iauthority_203 omitted because of collinearity
note: _Iauthority_205 omitted because of collinearity
note: _Iauthority_206 omitted because of collinearity
note: _Iauthority_207 omitted because of collinearity
note: _Iauthority_208 omitted because of collinearity
note: _Iauthority_209 omitted because of collinearity
note: _Iauthority_210 omitted because of collinearity
note: _Iauthority_211 omitted because of collinearity
note: _Iauthority_212 omitted because of collinearity
note: _Iauthority_213 omitted because of collinearity
note: _Iauthority_214 omitted because of collinearity
note: _Iauthority_215 omitted because of collinearity
note: _Iauthority_216 omitted because of collinearity
note: _Iauthority_217 omitted because of collinearity
note: _Iauthority_218 omitted because of collinearity
note: _Iauthority_219 omitted because of collinearity
note: _Iauthority_220 omitted because of collinearity
note: _Iauthority_221 omitted because of collinearity
note: _Iauthority_222 omitted because of collinearity
note: _Iauthority_223 omitted because of collinearity
note: _Iauthority_224 omitted because of collinearity
note: _Iauthority_225 omitted because of collinearity
note: _Iauthority_226 omitted because of collinearity
note: _Iauthority_227 omitted because of collinearity
note: _Iauthority_228 omitted because of collinearity
note: _Iauthority_229 omitted because of collinearity
note: _Iauthority_230 omitted because of collinearity
note: _Iauthority_231 omitted because of collinearity
note: _Iauthority_232 omitted because of collinearity
note: _Iauthority_233 omitted because of collinearity
note: _Iauthority_234 omitted because of collinearity
note: _Iauthority_235 omitted because of collinearity
note: _Iauthority_236 omitted because of collinearity
note: _Iauthority_237 omitted because of collinearity
note: _Iauthority_239 omitted because of collinearity
note: _Iauthority_240 omitted because of collinearity
note: _Iauthority_241 omitted because of collinearity
note: _Iauthority_242 omitted because of collinearity
note: _Iauthority_243 omitted because of collinearity
note: _Iauthority_244 omitted because of collinearity
note: _Iauthority_245 omitted because of collinearity
note: _Iauthority_246 omitted because of collinearity
note: _Iauthority_247 omitted because of collinearity
note: _Iauthority_248 omitted because of collinearity
note: _Iauthority_249 omitted because of collinearity
note: _Iauthority_250 omitted because of collinearity
note: _Iauthority_251 omitted because of collinearity
note: _Iauthority_252 omitted because of collinearity
note: _Iauthority_253 omitted because of collinearity
note: _Iauthority_254 omitted because of collinearity
note: _Iauthority_255 omitted because of collinearity
note: _Iauthority_256 omitted because of collinearity
note: _Iauthority_257 omitted because of collinearity
note: _Iauthority_258 omitted because of collinearity
note: _Iauthority_259 omitted because of collinearity
note: _Iauthority_260 omitted because of collinearity
note: _Iauthority_261 omitted because of collinearity
note: _Iauthority_262 omitted because of collinearity
note: _Iauthority_263 omitted because of collinearity
note: _Iauthority_264 omitted because of collinearity
note: _Iauthority_265 omitted because of collinearity
note: _Iauthority_266 omitted because of collinearity
note: _Iauthority_267 omitted because of collinearity
note: _Iauthority_268 omitted because of collinearity
note: _Iauthority_269 omitted because of collinearity
note: _Iauthority_270 omitted because of collinearity
note: _Iauthority_271 omitted because of collinearity
note: _Iauthority_272 omitted because of collinearity
note: _Iauthority_273 omitted because of collinearity
note: _Iauthority_274 omitted because of collinearity
note: _Iauthority_275 omitted because of collinearity
note: _Iauthority_276 omitted because of collinearity
note: _Iauthority_277 omitted because of collinearity
note: _Iauthority_278 omitted because of collinearity
note: _Iauthority_279 omitted because of collinearity
note: _Iauthority_280 omitted because of collinearity
note: _Iauthority_281 omitted because of collinearity
note: _Iauthority_282 omitted because of collinearity
note: _Iauthority_283 omitted because of collinearity
note: _Iauthority_284 omitted because of collinearity
note: _Iauthority_285 omitted because of collinearity
note: _Iauthority_286 omitted because of collinearity
note: _Iauthority_287 omitted because of collinearity
note: _Iauthority_288 omitted because of collinearity
note: _Iauthority_289 omitted because of collinearity
note: _Iauthority_290 omitted because of collinearity
note: _Iauthority_291 omitted because of collinearity
note: _Iauthority_292 omitted because of collinearity
note: _Iauthority_293 omitted because of collinearity
note: _Iauthority_294 omitted because of collinearity
note: _Iauthority_295 omitted because of collinearity
note: _Iauthority_296 omitted because of collinearity
note: _Iauthority_297 omitted because of collinearity
note: _Iauthority_298 omitted because of collinearity
note: _Iauthority_299 omitted because of collinearity
note: _Iauthority_300 omitted because of collinearity
note: _Iauthority_301 omitted because of collinearity
note: _Iauthority_302 omitted because of collinearity
note: _Iauthority_303 omitted because of collinearity
note: _Iauthority_304 omitted because of collinearity
note: _Iauthority_305 omitted because of collinearity
note: _Iauthority_306 omitted because of collinearity
note: _Iauthority_307 omitted because of collinearity
note: _Iauthority_308 omitted because of collinearity
note: _Iauthority_309 omitted because of collinearity
note: _Iauthority_310 omitted because of collinearity
note: _Iauthority_311 omitted because of collinearity
note: _Iauthority_312 omitted because of collinearity
note: _Iauthority_313 omitted because of collinearity
note: _Iauthority_314 omitted because of collinearity
note: _Iauthority_315 omitted because of collinearity
note: _Iauthority_316 omitted because of collinearity
note: _Iauthority_317 omitted because of collinearity
note: _Iauthority_318 omitted because of collinearity
note: _Iauthority_319 omitted because of collinearity
note: _Iauthority_320 omitted because of collinearity
note: _Iauthority_321 omitted because of collinearity
note: _Iauthority_322 omitted because of collinearity
note: _Iauthority_323 omitted because of collinearity
note: _Iauthority_324 omitted because of collinearity
note: _Iauthority_325 omitted because of collinearity
note: _Iauthority_326 omitted because of collinearity
note: _Iauthority_327 omitted because of collinearity
note: _Iauthority_328 omitted because of collinearity
note: _Iauthority_329 omitted because of collinearity
note: _Iauthority_330 omitted because of collinearity
note: _Iauthority_331 omitted because of collinearity
note: _Iauthority_332 omitted because of collinearity
note: _Iauthority_333 omitted because of collinearity
note: _Iauthority_334 omitted because of collinearity
note: _Iauthority_335 omitted because of collinearity
note: _Iauthority_336 omitted because of collinearity
note: _Iauthority_337 omitted because of collinearity
note: _Iauthority_338 omitted because of collinearity
note: _Iauthority_339 omitted because of collinearity
note: _Iauthority_340 omitted because of collinearity
note: _Iauthority_341 omitted because of collinearity
note: _Iauthority_342 omitted because of collinearity
note: _Iauthority_343 omitted because of collinearity
note: _Iauthority_344 omitted because of collinearity
note: _Iauthority_345 omitted because of collinearity
note: _Iauthority_346 omitted because of collinearity
note: _Iauthority_347 omitted because of collinearity
note: _Iauthority_348 omitted because of collinearity
note: _Iauthority_349 omitted because of collinearity
note: _Iauthority_350 omitted because of collinearity
note: _Iauthority_351 omitted because of collinearity
note: _Iauthority_352 omitted because of collinearity
note: _Iauthority_353 omitted because of collinearity
note: _Iauthority_354 omitted because of collinearity
note: _Iauthority_355 omitted because of collinearity
note: _Iauthority_356 omitted because of collinearity
note: _Iauthority_357 omitted because of collinearity
note: _Iauthority_358 omitted because of collinearity
note: _Iauthority_359 omitted because of collinearity
note: _Iauthority_360 omitted because of collinearity
note: _Iauthority_361 omitted because of collinearity
note: _Iauthority_362 omitted because of collinearity
note: _Iauthority_363 omitted because of collinearity
note: _Iauthority_364 omitted because of collinearity
note: _Iauthority_365 omitted because of collinearity
note: _Iauthority_366 omitted because of collinearity
note: _Iauthority_367 omitted because of collinearity
note: _Iauthority_368 omitted because of collinearity
note: _Iauthority_369 omitted because of collinearity
note: _Iauthority_370 omitted because of collinearity
note: _Iauthority_371 omitted because of collinearity
note: _Iauthority_373 omitted because of collinearity
note: _Iauthority_374 omitted because of collinearity
note: _Iauthority_375 omitted because of collinearity
note: _Iauthority_376 omitted because of collinearity
note: _Iauthority_377 omitted because of collinearity
note: _Iauthority_378 omitted because of collinearity
note: _Iauthority_379 omitted because of collinearity
note: _Iauthority_380 omitted because of collinearity
note: _Iauthority_381 omitted because of collinearity
note: _Iauthority_382 omitted because of collinearity
note: _Iauthority_383 omitted because of collinearity
note: _Iauthority_384 omitted because of collinearity
note: _Iauthority_385 omitted because of collinearity
note: _Iauthority_386 omitted because of collinearity
note: _Iauthority_387 omitted because of collinearity
note: _Iauthority_388 omitted because of collinearity
note: _Iauthority_389 omitted because of collinearity
note: _Iauthority_390 omitted because of collinearity
note: _Iauthority_391 omitted because of collinearity
note: _Iauthority_392 omitted because of collinearity
note: _Iauthority_393 omitted because of collinearity
note: _Iauthority_394 omitted because of collinearity
note: _Iauthority_395 omitted because of collinearity
note: _Iauthority_396 omitted because of collinearity
note: _Iauthority_397 omitted because of collinearity
note: _Iauthority_398 omitted because of collinearity
note: _Iauthority_399 omitted because of collinearity
note: _Iauthority_400 omitted because of collinearity

note: _Iauthority_401 omitted because of collinearity
note: _Iauthority_402 omitted because of collinearity
note: _Iauthority_403 omitted because of collinearity
note: _Iauthority_404 omitted because of collinearity
note: _Iauthority_405 omitted because of collinearity
note: _Iauthority_406 omitted because of collinearity
note: _Iauthority_407 omitted because of collinearity
note: _Iauthority_408 omitted because of collinearity
note: _Iauthority_409 omitted because of collinearity
note: _Iauthority_410 omitted because of collinearity
note: _Iauthority_411 omitted because of collinearity
note: _Iauthority_412 omitted because of collinearity
note: _Iauthority_413 omitted because of collinearity
note: _Iauthority_414 omitted because of collinearity
note: _Iauthority_415 omitted because of collinearity
note: _Iauthority_416 omitted because of collinearity
note: _Iauthority_417 omitted because of collinearity
note: _Iauthority_418 omitted because of collinearity
note: _Iauthority_419 omitted because of collinearity
note: _Iauthority_420 omitted because of collinearity
note: _Iauthority_421 omitted because of collinearity
note: _Iauthority_422 omitted because of collinearity
note: _Iauthority_423 omitted because of collinearity
note: _Iauthority_424 omitted because of collinearity
note: _Iauthority_425 omitted because of collinearity
note: _Iauthority_426 omitted because of collinearity
note: _Iauthority_427 omitted because of collinearity
note: _Iauthority_428 omitted because of collinearity
note: _Iauthority_429 omitted because of collinearity
note: _Iauthority_430 omitted because of collinearity
note: _Iauthority_431 omitted because of collinearity
note: _Iauthority_432 omitted because of collinearity
note: _Iauthority_433 omitted because of collinearity
note: _Iauthority_434 omitted because of collinearity
note: _Iauthority_435 omitted because of collinearity
note: _Iauthority_436 omitted because of collinearity
note: _Iauthority_437 omitted because of collinearity
note: _Iauthority_438 omitted because of collinearity
note: _Iauthority_439 omitted because of collinearity
note: _Iauthority_440 omitted because of collinearity
note: _Iauthority_441 omitted because of collinearity
note: _Iauthority_442 omitted because of collinearity
note: _Iauthority_443 omitted because of collinearity
note: _Iauthority_444 omitted because of collinearity
note: _Iauthority_445 omitted because of collinearity
note: _Iauthority_446 omitted because of collinearity
note: _Iauthority_447 omitted because of collinearity
note: _Iauthority_448 omitted because of collinearity
note: _Iauthority_449 omitted because of collinearity
note: _Iauthority_450 omitted because of collinearity
note: _Iauthority_451 omitted because of collinearity
note: _Iauthority_452 omitted because of collinearity
note: _Iauthority_453 omitted because of collinearity
note: _Iauthority_454 omitted because of collinearity
note: _Iauthority_455 omitted because of collinearity
note: _Iauthority_456 omitted because of collinearity
note: _Iauthority_457 omitted because of collinearity
note: _Iauthority_458 omitted because of collinearity
note: _Iauthority_459 omitted because of collinearity
note: _Iauthority_460 omitted because of collinearity
note: _Iauthority_461 omitted because of collinearity
note: _Iauthority_462 omitted because of collinearity
note: _Iauthority_463 omitted because of collinearity
note: _Iauthority_464 omitted because of collinearity
note: _Iauthority_465 omitted because of collinearity
note: _Iauthority_466 omitted because of collinearity
note: _Iauthority_467 omitted because of collinearity
note: _Iauthority_468 omitted because of collinearity
note: _Iauthority_469 omitted because of collinearity
note: _Iauthority_470 omitted because of collinearity
note: _Iauthority_471 omitted because of collinearity
note: _Iauthority_472 omitted because of collinearity
note: _Iauthority_473 omitted because of collinearity
note: _Iauthority_474 omitted because of collinearity
note: _Iauthority_475 omitted because of collinearity
note: _Iauthority_476 omitted because of collinearity
note: _Iauthority_477 omitted because of collinearity
note: _Iauthority_478 omitted because of collinearity
note: _Iauthority_479 omitted because of collinearity
note: _Iauthority_480 omitted because of collinearity
note: _Iauthority_481 omitted because of collinearity
note: _Iauthority_482 omitted because of collinearity
note: _Iauthority_483 omitted because of collinearity
note: _Iauthority_484 omitted because of collinearity
note: _Iauthority_485 omitted because of collinearity
note: _Iauthority_486 omitted because of collinearity
note: _Iauthority_487 omitted because of collinearity
note: _Iauthority_488 omitted because of collinearity
note: _Iauthority_489 omitted because of collinearity
note: _Iauthority_490 omitted because of collinearity
note: _Iauthority_491 omitted because of collinearity
note: _Iauthority_492 omitted because of collinearity
note: _Iauthority_493 omitted because of collinearity
note: _Iauthority_494 omitted because of collinearity
note: _Iauthority_495 omitted because of collinearity
note: _Iauthority_496 omitted because of collinearity
note: _Iauthority_497 omitted because of collinearity
note: _Iauthority_498 omitted because of collinearity
note: _Iauthority_499 omitted because of collinearity
note: _Iauthority_500 omitted because of collinearity

note: _Iauthority_501 omitted because of collinearity
note: _Iauthority_502 omitted because of collinearity
note: _Iauthority_503 omitted because of collinearity
note: _Iauthority_504 omitted because of collinearity
note: _Iauthority_505 omitted because of collinearity
note: _Iauthority_506 omitted because of collinearity
note: _Iauthority_507 omitted because of collinearity
note: _Iauthority_508 omitted because of collinearity
note: _Iauthority_509 omitted because of collinearity
note: _Iauthority_510 omitted because of collinearity
note: _Iauthority_511 omitted because of collinearity
note: _Iauthority_512 omitted because of collinearity
note: _Iauthority_513 omitted because of collinearity
note: _Iauthority_514 omitted because of collinearity
note: _Iauthority_515 omitted because of collinearity
note: _Iauthority_516 omitted because of collinearity
note: _Iauthority_517 omitted because of collinearity
note: _Iauthority_518 omitted because of collinearity
note: _Iauthority_519 omitted because of collinearity
note: _Iauthority_520 omitted because of collinearity
note: _Iauthority_521 omitted because of collinearity
note: _Iauthority_522 omitted because of collinearity
note: _Iauthority_523 omitted because of collinearity
note: _Iauthority_524 omitted because of collinearity
note: _Iauthority_525 omitted because of collinearity
note: _Iauthority_526 omitted because of collinearity
note: _Iauthority_527 omitted because of collinearity
note: _Iauthority_528 omitted because of collinearity
note: _Iauthority_529 omitted because of collinearity
note: _Iauthority_530 omitted because of collinearity
note: _Iauthority_531 omitted because of collinearity
note: _Iauthority_532 omitted because of collinearity
note: _Iauthority_533 omitted because of collinearity
note: _Iauthority_534 omitted because of collinearity
note: _Iauthority_535 omitted because of collinearity
note: _Iauthority_536 omitted because of collinearity
note: _Iauthority_537 omitted because of collinearity
note: _Iauthority_538 omitted because of collinearity
note: _Iauthority_539 omitted because of collinearity
note: _Iauthority_540 omitted because of collinearity
note: _Iauthority_541 omitted because of collinearity
note: _Iauthority_542 omitted because of collinearity
note: _Iauthority_543 omitted because of collinearity
note: _Iauthority_544 omitted because of collinearity
note: _Iauthority_545 omitted because of collinearity
note: _Iauthority_546 omitted because of collinearity
note: _Iauthority_547 omitted because of collinearity
note: _Iauthority_548 omitted because of collinearity
note: _Iauthority_549 omitted because of collinearity
note: _Iauthority_550 omitted because of collinearity
note: _Iauthority_551 omitted because of collinearity
note: _Iauthority_552 omitted because of collinearity
note: _Iauthority_553 omitted because of collinearity
note: _Iauthority_554 omitted because of collinearity
note: _Iauthority_555 omitted because of collinearity
note: _Iauthority_556 omitted because of collinearity
note: _Iauthority_557 omitted because of collinearity
note: _Iauthority_558 omitted because of collinearity
note: _Iauthority_559 omitted because of collinearity
note: _Iauthority_560 omitted because of collinearity
note: _Iauthority_561 omitted because of collinearity
note: _Iauthority_562 omitted because of collinearity
note: _Iauthority_563 omitted because of collinearity
note: _Iauthority_564 omitted because of collinearity

note: _Iauthority_566 omitted because of collinearity
note: _Iauthority_567 omitted because of collinearity
note: _Iauthority_568 omitted because of collinearity
note: _Iauthority_569 omitted because of collinearity
note: _Iauthority_570 omitted because of collinearity
note: _Iauthority_571 omitted because of collinearity
note: _Iauthority_572 omitted because of collinearity
note: _Iauthority_573 omitted because of collinearity
note: _Iauthority_574 omitted because of collinearity
note: _Iauthority_575 omitted because of collinearity
note: _Iauthority_576 omitted because of collinearity
note: _Iauthority_577 omitted because of collinearity
note: _Iauthority_578 omitted because of collinearity
note: _Iauthority_579 omitted because of collinearity
note: _Iauthority_580 omitted because of collinearity
note: _Iauthority_581 omitted because of collinearity
note: _Iauthority_582 omitted because of collinearity
note: _Iauthority_583 omitted because of collinearity
note: _Iauthority_584 omitted because of collinearity
note: _Iauthority_585 omitted because of collinearity
note: _Iauthority_586 omitted because of collinearity
note: _Iauthority_587 omitted because of collinearity
note: _Iauthority_588 omitted because of collinearity
note: _Iauthority_589 omitted because of collinearity
note: _Iauthority_590 omitted because of collinearity
note: _Iauthority_591 omitted because of collinearity
note: _Iauthority_592 omitted because of collinearity
note: _Iauthority_593 omitted because of collinearity
note: _Iauthority_594 omitted because of collinearity
note: _Iauthority_596 omitted because of collinearity
note: _Iauthority_597 omitted because of collinearity
note: _Iauthority_598 omitted because of collinearity
note: _Iauthority_599 omitted because of collinearity
note: _Iauthority_600 omitted because of collinearity
note: _Iauthority_601 omitted because of collinearity
note: _Iauthority_602 omitted because of collinearity
note: _Iauthority_603 omitted because of collinearity
note: _Iauthority_604 omitted because of collinearity
note: _Iauthority_605 omitted because of collinearity
note: _Iauthority_606 omitted because of collinearity
note: _Iauthority_607 omitted because of collinearity
note: _Iauthority_608 omitted because of collinearity
note: _Iauthority_609 omitted because of collinearity
note: _Iauthority_610 omitted because of collinearity
note: _Iauthority_611 omitted because of collinearity
note: _Iauthority_612 omitted because of collinearity
note: _Iauthority_613 omitted because of collinearity
note: _Iauthority_614 omitted because of collinearity
note: _Iauthority_615 omitted because of collinearity
note: _Iauthority_616 omitted because of collinearity
note: _Iauthority_617 omitted because of collinearity
note: _Iauthority_618 omitted because of collinearity
note: _Iauthority_619 omitted because of collinearity
note: _Iauthority_620 omitted because of collinearity
note: _Iauthority_621 omitted because of collinearity
note: _Iauthority_623 omitted because of collinearity
note: _Iauthority_624 omitted because of collinearity
note: _Iauthority_625 omitted because of collinearity
note: _Iauthority_626 omitted because of collinearity
note: _Iauthority_627 omitted because of collinearity
note: _Iauthority_628 omitted because of collinearity
note: _Iauthority_629 omitted because of collinearity
note: _Iauthority_630 omitted because of collinearity
note: _Iauthority_631 omitted because of collinearity
note: _Iauthority_632 omitted because of collinearity
note: _Iauthority_633 omitted because of collinearity
note: _Iauthority_634 omitted because of collinearity
note: _Iauthority_635 omitted because of collinearity
note: _Iauthority_636 omitted because of collinearity
note: _Iauthority_637 omitted because of collinearity
note: _Iauthority_638 omitted because of collinearity
note: _Iauthority_639 omitted because of collinearity
note: _Iauthority_640 omitted because of collinearity
note: _Iauthority_641 omitted because of collinearity
note: _Iauthority_642 omitted because of collinearity
note: _Iauthority_643 omitted because of collinearity
note: _Iauthority_644 omitted because of collinearity
note: _Iauthority_645 omitted because of collinearity
note: _Iauthority_646 omitted because of collinearity
note: _Iauthority_647 omitted because of collinearity
note: _Iauthority_648 omitted because of collinearity
note: _Iauthority_649 omitted because of collinearity
note: _Iauthority_650 omitted because of collinearity
note: _Iauthority_651 omitted because of collinearity
note: _Iauthority_652 omitted because of collinearity
note: _Iauthority_653 omitted because of collinearity
note: _Iauthority_654 omitted because of collinearity
note: _Iauthority_655 omitted because of collinearity
note: _Iauthority_656 omitted because of collinearity
note: _Iauthority_657 omitted because of collinearity
note: _Iauthority_658 omitted because of collinearity
note: _Iauthority_659 omitted because of collinearity
note: _Iauthority_660 omitted because of collinearity
note: _Iauthority_661 omitted because of collinearity
note: _Iauthority_662 omitted because of collinearity
note: _Iauthority_663 omitted because of collinearity
note: _Iauthority_664 omitted because of collinearity
note: _Iauthority_665 omitted because of collinearity
note: _Iauthority_666 omitted because of collinearity
note: _Iauthority_667 omitted because of collinearity
note: _Iauthority_668 omitted because of collinearity
note: _Iauthority_669 omitted because of collinearity
note: _Iauthority_670 omitted because of collinearity
note: _Iauthority_671 omitted because of collinearity
note: _Iauthority_672 omitted because of collinearity
note: _Iauthority_673 omitted because of collinearity
note: _Iauthority_674 omitted because of collinearity
note: _Iauthority_675 omitted because of collinearity
note: _Iauthority_676 omitted because of collinearity
note: _Iauthority_677 omitted because of collinearity
note: _Iauthority_678 omitted because of collinearity
note: _Iauthority_679 omitted because of collinearity
note: _Iauthority_680 omitted because of collinearity
note: _Iauthority_681 omitted because of collinearity
note: _Iauthority_682 omitted because of collinearity
note: _Iauthority_683 omitted because of collinearity
note: _Iauthority_684 omitted because of collinearity
note: _Iauthority_685 omitted because of collinearity
note: _Iauthority_686 omitted because of collinearity
note: _Iauthority_687 omitted because of collinearity
note: _Iauthority_688 omitted because of collinearity
note: _Iauthority_689 omitted because of collinearity
note: _Iauthority_690 omitted because of collinearity
note: _Iauthority_691 omitted because of collinearity
note: _Iauthority_692 omitted because of collinearity
note: _Iauthority_693 omitted because of collinearity
note: _Iauthority_694 omitted because of collinearity
note: _Iauthority_695 omitted because of collinearity
note: _Iauthority_696 omitted because of collinearity
note: _Iauthority_697 omitted because of collinearity
note: _Iauthority_698 omitted because of collinearity
note: _Iauthority_699 omitted because of collinearity
note: _Iauthority_700 omitted because of collinearity
note: _Iauthority_701 omitted because of collinearity
note: _Iauthority_702 omitted because of collinearity
note: _Iauthority_703 omitted because of collinearity
note: _Iauthority_704 omitted because of collinearity
note: _Iauthority_705 omitted because of collinearity
note: _Iauthority_706 omitted because of collinearity
note: _Iauthority_707 omitted because of collinearity
note: _Iauthority_708 omitted because of collinearity
note: _Iauthority_709 omitted because of collinearity
note: _Iauthority_710 omitted because of collinearity
note: _Iauthority_711 omitted because of collinearity
note: _Iauthority_712 omitted because of collinearity
note: _Iauthority_713 omitted because of collinearity
note: _Iauthority_714 omitted because of collinearity
note: _Iauthority_715 omitted because of collinearity
note: _Iauthority_716 omitted because of collinearity
note: _Iauthority_717 omitted because of collinearity
note: _Iauthority_718 omitted because of collinearity
note: _Iauthority_719 omitted because of collinearity
note: _Iauthority_720 omitted because of collinearity
note: _Iauthority_721 omitted because of collinearity
note: _Iauthority_722 omitted because of collinearity
note: _Iauthority_723 omitted because of collinearity
note: _Iauthority_724 omitted because of collinearity
note: _Iauthority_725 omitted because of collinearity
note: _Iauthority_726 omitted because of collinearity
note: _Iauthority_727 omitted because of collinearity
note: _Iauthority_728 omitted because of collinearity
note: _Iauthority_729 omitted because of collinearity
note: _Iauthority_730 omitted because of collinearity
note: _Iauthority_731 omitted because of collinearity
note: _Iauthority_732 omitted because of collinearity
note: _Iauthority_733 omitted because of collinearity
note: _Iauthority_734 omitted because of collinearity
note: _Iauthority_735 omitted because of collinearity
note: _Iauthority_736 omitted because of collinearity
note: _Iauthority_737 omitted because of collinearity
note: _Iauthority_738 omitted because of collinearity
note: _Iauthority_739 omitted because of collinearity
note: _Iauthority_740 omitted because of collinearity
note: _Iauthority_741 omitted because of collinearity
note: _Iauthority_742 omitted because of collinearity
note: _Iauthority_743 omitted because of collinearity
note: _Iauthority_744 omitted because of collinearity
note: _Iauthority_745 omitted because of collinearity
note: _Iauthority_746 omitted because of collinearity
note: _Iauthority_747 omitted because of collinearity
note: _Iauthority_748 omitted because of collinearity
note: _Iauthority_749 omitted because of collinearity
note: _Iauthority_750 omitted because of collinearity
note: _Iauthority_751 omitted because of collinearity
note: _Iauthority_752 omitted because of collinearity
note: _Iauthority_753 omitted because of collinearity
note: _Iauthority_754 omitted because of collinearity
note: _Iauthority_755 omitted because of collinearity
note: _Iauthority_756 omitted because of collinearity
note: _Iauthority_757 omitted because of collinearity
note: _Iauthority_758 omitted because of collinearity
note: _Iauthority_759 omitted because of collinearity
note: _Iauthority_760 omitted because of collinearity
note: _Iauthority_761 omitted because of collinearity
note: _Iauthority_762 omitted because of collinearity
note: _Iauthority_763 omitted because of collinearity
note: _Iauthority_764 omitted because of collinearity
note: _Iauthority_765 omitted because of collinearity
note: _Iauthority_766 omitted because of collinearity
note: _Iauthority_767 omitted because of collinearity
note: _Iauthority_768 omitted because of collinearity
note: _Iauthority_769 omitted because of collinearity
note: _Iauthority_770 omitted because of collinearity
note: _Iauthority_771 omitted because of collinearity
note: _Iauthority_772 omitted because of collinearity
note: _Iauthority_773 omitted because of collinearity
note: _Iauthority_774 omitted because of collinearity
note: _Iauthority_775 omitted because of collinearity
note: _Iauthority_776 omitted because of collinearity
note: _Iauthority_777 omitted because of collinearity
note: _Iauthority_778 omitted because of collinearity
note: _Iauthority_779 omitted because of collinearity
note: _Iauthority_780 omitted because of collinearity
note: _Iauthority_781 omitted because of collinearity
note: _Iauthority_782 omitted because of collinearity
note: _Iauthority_783 omitted because of collinearity
note: _Iauthority_784 omitted because of collinearity
note: _Iauthority_785 omitted because of collinearity
note: _Iauthority_786 omitted because of collinearity
note: _Iauthority_787 omitted because of collinearity
note: _Iauthority_788 omitted because of collinearity
note: _Iauthority_789 omitted because of collinearity
note: _Iauthority_790 omitted because of collinearity
note: _Iauthority_791 omitted because of collinearity
note: _Iauthority_792 omitted because of collinearity
note: _Iauthority_793 omitted because of collinearity
note: _Iauthority_794 omitted because of collinearity
note: _Iauthority_795 omitted because of collinearity
note: _Iauthority_796 omitted because of collinearity
note: _Iauthority_797 omitted because of collinearity
note: _Iauthority_798 omitted because of collinearity
note: _Iauthority_799 omitted because of collinearity
note: _Iauthority_800 omitted because of collinearity
note: _Iauthority_801 omitted because of collinearity
note: _Iauthority_802 omitted because of collinearity
note: _Iauthority_803 omitted because of collinearity
note: _Iauthority_804 omitted because of collinearity
note: _Iauthority_805 omitted because of collinearity
note: _Iauthority_806 omitted because of collinearity
note: _Iauthority_807 omitted because of collinearity
note: _Iauthority_808 omitted because of collinearity
note: _Iauthority_809 omitted because of collinearity
note: _Iauthority_810 omitted because of collinearity
note: _Iauthority_811 omitted because of collinearity
note: _Iauthority_812 omitted because of collinearity
note: _Iauthority_813 omitted because of collinearity
note: _Iauthority_814 omitted because of collinearity
note: _Iauthority_815 omitted because of collinearity
note: _Iauthority_816 omitted because of collinearity
note: _Iauthority_817 omitted because of collinearity
note: _Iauthority_818 omitted because of collinearity
note: _Iauthority_819 omitted because of collinearity
note: _Iauthority_820 omitted because of collinearity
note: _Iauthority_821 omitted because of collinearity
note: _Iauthority_822 omitted because of collinearity
note: _Iauthority_823 omitted because of collinearity
note: _Iauthority_824 omitted because of collinearity
note: _Iauthority_825 omitted because of collinearity
note: _Iauthority_826 omitted because of collinearity
note: _Iauthority_827 omitted because of collinearity
note: _Iauthority_828 omitted because of collinearity
note: _Iauthority_829 omitted because of collinearity
note: _Iauthority_830 omitted because of collinearity
note: _Iauthority_831 omitted because of collinearity
note: _Iauthority_832 omitted because of collinearity
note: _Iauthority_833 omitted because of collinearity
note: _Iauthority_834 omitted because of collinearity
note: _Iauthority_835 omitted because of collinearity
note: _Iauthority_836 omitted because of collinearity
note: _Iauthority_837 omitted because of collinearity
note: _Iauthority_838 omitted because of collinearity
note: _Iauthority_839 omitted because of collinearity
note: _Iauthority_840 omitted because of collinearity
note: _Iauthority_841 omitted because of collinearity
note: _Iauthority_842 omitted because of collinearity
note: _Iauthority_843 omitted because of collinearity
note: _Iauthority_844 omitted because of collinearity
note: _Iauthority_845 omitted because of collinearity
note: _Iauthority_846 omitted because of collinearity
note: _Iauthority_847 omitted because of collinearity
note: _Iauthority_848 omitted because of collinearity
note: _Iauthority_849 omitted because of collinearity
note: _Iauthority_850 omitted because of collinearity
note: _Iauthority_851 omitted because of collinearity
note: _Iauthority_852 omitted because of collinearity
note: _Iauthority_853 omitted because of collinearity
note: _Iauthority_854 omitted because of collinearity
note: _Iauthority_855 omitted because of collinearity
note: _Iauthority_856 omitted because of collinearity
note: _Iauthority_857 omitted because of collinearity
note: _Iauthority_858 omitted because of collinearity
note: _Iauthority_859 omitted because of collinearity
note: _Iauthority_860 omitted because of collinearity
note: _Iauthority_861 omitted because of collinearity
note: _Iauthority_862 omitted because of collinearity
note: _Iauthority_863 omitted because of collinearity
note: _Iauthority_864 omitted because of collinearity
note: _Iauthority_866 omitted because of collinearity
note: _Iauthority_867 omitted because of collinearity
note: _Iauthority_868 omitted because of collinearity
note: _Iauthority_869 omitted because of collinearity
note: _Iauthority_870 omitted because of collinearity
note: _Iauthority_871 omitted because of collinearity
note: _Iauthority_872 omitted because of collinearity
note: _Iauthority_873 omitted because of collinearity
note: _Iauthority_874 omitted because of collinearity
note: _Iauthority_875 omitted because of collinearity
note: _Iauthority_876 omitted because of collinearity
note: _Iauthority_877 omitted because of collinearity
note: _Iauthority_878 omitted because of collinearity
note: _Iauthority_879 omitted because of collinearity
note: _Iauthority_880 omitted because of collinearity
note: _Iauthority_881 omitted because of collinearity
note: _Iauthority_882 omitted because of collinearity
note: _Iauthority_883 omitted because of collinearity
note: _Iauthority_884 omitted because of collinearity
note: _Iauthority_885 omitted because of collinearity
note: _Iauthority_886 omitted because of collinearity
note: _Iauthority_887 omitted because of collinearity
note: _Iauthority_888 omitted because of collinearity
note: _Iauthority_889 omitted because of collinearity
note: _Iauthority_890 omitted because of collinearity
note: _Iauthority_891 omitted because of collinearity
note: _Iauthority_892 omitted because of collinearity
note: _Iauthority_893 omitted because of collinearity
note: _Iauthority_894 omitted because of collinearity
note: _Iauthority_895 omitted because of collinearity
note: _Iauthority_896 omitted because of collinearity
note: _Iauthority_897 omitted because of collinearity
note: _Iauthority_898 omitted because of collinearity
note: _Iauthority_899 omitted because of collinearity
note: _Iauthority_900 omitted because of collinearity
note: _Iauthority_901 omitted because of collinearity
note: _Iauthority_902 omitted because of collinearity
note: _Iauthority_903 omitted because of collinearity
note: _Iauthority_904 omitted because of collinearity
note: _Iauthority_905 omitted because of collinearity
note: _Iauthority_906 omitted because of collinearity
note: _Iauthority_907 omitted because of collinearity
note: _Iauthority_908 omitted because of collinearity
note: _Iauthority_910 omitted because of collinearity
note: _Iauthority_911 omitted because of collinearity
note: _Iauthority_912 omitted because of collinearity
note: _Iauthority_913 omitted because of collinearity
note: _Iauthority_914 omitted because of collinearity
note: _Iauthority_915 omitted because of collinearity
note: _Iauthority_916 omitted because of collinearity
note: _Iauthority_917 omitted because of collinearity
note: _Iauthority_918 omitted because of collinearity
note: _Iauthority_919 omitted because of collinearity
note: _Iauthority_921 omitted because of collinearity
note: _Iauthority_922 omitted because of collinearity
note: _Iauthority_923 omitted because of collinearity
note: _Iauthority_924 omitted because of collinearity
note: _Iauthority_925 omitted because of collinearity
note: _Iauthority_926 omitted because of collinearity
note: _Iauthority_927 omitted because of collinearity
note: _Iauthority_928 omitted because of collinearity
note: _Iauthority_929 omitted because of collinearity
note: _Iauthority_930 omitted because of collinearity
note: _Iauthority_931 omitted because of collinearity
note: _Iauthority_932 omitted because of collinearity
note: _Iauthority_933 omitted because of collinearity
note: _Iauthority_934 omitted because of collinearity
note: _Iauthority_935 omitted because of collinearity
note: _Iauthority_936 omitted because of collinearity
note: _Iauthority_937 omitted because of collinearity
note: _Iauthority_938 omitted because of collinearity
note: _Iauthority_939 omitted because of collinearity
note: _Iauthority_940 omitted because of collinearity
note: _Iauthority_941 omitted because of collinearity
note: _Iauthority_942 omitted because of collinearity
note: _Iauthority_943 omitted because of collinearity
note: _Iauthority_944 omitted because of collinearity
note: _Iauthority_945 omitted because of collinearity
note: _Iauthority_946 omitted because of collinearity
note: _Iauthority_947 omitted because of collinearity
note: _Iauthority_948 omitted because of collinearity
note: _Iauthority_949 omitted because of collinearity
note: _Iauthority_950 omitted because of collinearity
note: _Iauthority_951 omitted because of collinearity
note: _Iauthority_952 omitted because of collinearity
note: _Iauthority_953 omitted because of collinearity
note: _Iauthority_954 omitted because of collinearity
note: _Iauthority_955 omitted because of collinearity
note: _Iauthority_956 omitted because of collinearity
note: _Iauthority_957 omitted because of collinearity
note: _Iauthority_958 omitted because of collinearity
note: _Iauthority_959 omitted because of collinearity
note: _Iauthority_960 omitted because of collinearity
note: _Iauthority_961 omitted because of collinearity
note: _Iauthority_962 omitted because of collinearity
note: _Iauthority_963 omitted because of collinearity
note: _Iauthority_964 omitted because of collinearity
note: _Iauthority_965 omitted because of collinearity
note: _Iauthority_966 omitted because of collinearity
note: _Iauthority_967 omitted because of collinearity
note: _Iauthority_968 omitted because of collinearity
note: _Iauthority_969 omitted because of collinearity
note: _Iauthority_970 omitted because of collinearity
note: _Iauthority_971 omitted because of collinearity
note: _Iauthority_972 omitted because of collinearity
note: _Iauthority_973 omitted because of collinearity
note: _Iauthority_974 omitted because of collinearity
note: _Iauthority_975 omitted because of collinearity
note: _Iauthority_976 omitted because of collinearity
note: _Iauthority_977 omitted because of collinearity
note: _Iauthority_978 omitted because of collinearity
note: _Iauthority_979 omitted because of collinearity
note: _Iauthority_980 omitted because of collinearity
note: _Iauthority_981 omitted because of collinearity
note: _Iauthority_982 omitted because of collinearity
note: _Iauthority_983 omitted because of collinearity
note: _Iauthority_984 omitted because of collinearity
note: _Iauthority_985 omitted because of collinearity
note: _Iauthority_986 omitted because of collinearity
note: _Iauthority_988 omitted because of collinearity
note: _Iauthority_989 omitted because of collinearity
note: _Iauthority_990 omitted because of collinearity
note: _Iauthority_991 omitted because of collinearity
note: _Iauthority_992 omitted because of collinearity
note: _Iauthority_993 omitted because of collinearity
note: _Iauthority_994 omitted because of collinearity
note: _Iauthority_995 omitted because of collinearity
note: _Iauthority_996 omitted because of collinearity
note: _Iauthority_997 omitted because of collinearity
note: _Iauthority_998 omitted because of collinearity
note: _Iauthority_999 omitted because of collinearity
note: _Iauthority_1000 omitted because of collinearity
note: _Iauthority_1001 omitted because of collinearity
note: _Iauthority_1002 omitted because of collinearity
note: _Iauthority_1003 omitted because of collinearity
note: _Iauthority_1004 omitted because of collinearity
note: _Iauthority_1005 omitted because of collinearity
note: _Iauthority_1006 omitted because of collinearity
note: _Iauthority_1007 omitted because of collinearity
note: _Iauthority_1008 omitted because of collinearity
note: _Iauthority_1009 omitted because of collinearity
note: _Iauthority_1010 omitted because of collinearity
note: _Iauthority_1011 omitted because of collinearity
note: _Iauthority_1012 omitted because of collinearity
note: _Iauthority_1013 omitted because of collinearity
note: _Iauthority_1014 omitted because of collinearity
note: _Iauthority_1015 omitted because of collinearity
note: _Iauthority_1016 omitted because of collinearity
note: _Iauthority_1017 omitted because of collinearity
note: _Iauthority_1018 omitted because of collinearity
note: _Iauthority_1019 omitted because of collinearity
note: _Iauthority_1020 omitted because of collinearity
note: _Iauthority_1021 omitted because of collinearity
note: _Iauthority_1022 omitted because of collinearity
note: _Iauthority_1023 omitted because of collinearity
note: _Iauthority_1024 omitted because of collinearity
note: _Iauthority_1025 omitted because of collinearity
note: _Iauthority_1026 omitted because of collinearity
note: _Iauthority_1027 omitted because of collinearity
note: _Iauthority_1028 omitted because of collinearity
note: _Iauthority_1029 omitted because of collinearity
note: _Iauthority_1030 omitted because of collinearity
note: _Iauthority_1031 omitted because of collinearity
note: _Iauthority_1032 omitted because of collinearity
note: _Iauthority_1033 omitted because of collinearity
note: _Iauthority_1034 omitted because of collinearity
note: _Iauthority_1035 omitted because of collinearity
note: _Iauthority_1036 omitted because of collinearity
note: _Iauthority_1037 omitted because of collinearity
note: _Iauthority_1038 omitted because of collinearity
note: _Iauthority_1039 omitted because of collinearity
note: _Iauthority_1040 omitted because of collinearity
note: _Iauthority_1041 omitted because of collinearity
note: _Iauthority_1042 omitted because of collinearity
note: _Iauthority_1043 omitted because of collinearity
note: _Iauthority_1044 omitted because of collinearity
note: _Iauthority_1045 omitted because of collinearity
note: _Iauthority_1046 omitted because of collinearity
note: _Iauthority_1047 omitted because of collinearity
note: _Iauthority_1048 omitted because of collinearity
note: _Iauthority_1049 omitted because of collinearity
note: _Iauthority_1050 omitted because of collinearity
note: _Iauthority_1051 omitted because of collinearity
note: _Iauthority_1052 omitted because of collinearity
note: _Iauthority_1053 omitted because of collinearity
note: _Iauthority_1054 omitted because of collinearity
note: _Iauthority_1055 omitted because of collinearity
note: _Iauthority_1056 omitted because of collinearity
note: _Iauthority_1057 omitted because of collinearity
note: _Iauthority_1058 omitted because of collinearity
note: _Iauthority_1059 omitted because of collinearity
note: _Iauthority_1060 omitted because of collinearity
note: _Iauthority_1061 omitted because of collinearity
note: _Iauthority_1062 omitted because of collinearity
note: _Iauthority_1063 omitted because of collinearity
note: _Iauthority_1064 omitted because of collinearity
note: _Iauthority_1065 omitted because of collinearity
note: _Iauthority_1066 omitted because of collinearity
note: _Iauthority_1067 omitted because of collinearity
note: _Iauthority_1068 omitted because of collinearity
note: _Iauthority_1069 omitted because of collinearity
note: _Iauthority_1070 omitted because of collinearity
note: _Iauthority_1071 omitted because of collinearity
note: _Iauthority_1072 omitted because of collinearity
note: _Iauthority_1073 omitted because of collinearity
note: _Iauthority_1074 omitted because of collinearity
note: _Iauthority_1075 omitted because of collinearity
note: _Iauthority_1076 omitted because of collinearity
note: _Iauthority_1077 omitted because of collinearity
note: _Iauthority_1078 omitted because of collinearity
note: _Iauthority_1079 omitted because of collinearity
note: _Iauthority_1080 omitted because of collinearity
note: _Iauthority_1081 omitted because of collinearity
note: _Iauthority_1082 omitted because of collinearity
note: _Iauthority_1083 omitted because of collinearity
note: _Iauthority_1084 omitted because of collinearity
note: _Iauthority_1085 omitted because of collinearity
note: _Iauthority_1086 omitted because of collinearity
note: _Iauthority_1087 omitted because of collinearity
note: _Iauthority_1088 omitted because of collinearity
note: _Iauthority_1089 omitted because of collinearity
note: _Iauthority_1090 omitted because of collinearity
note: _Iauthority_1091 omitted because of collinearity
note: _Iauthority_1092 omitted because of collinearity
note: _Iauthority_1093 omitted because of collinearity
note: _Iauthority_1094 omitted because of collinearity
note: _Iauthority_1095 omitted because of collinearity
note: _Iauthority_1096 omitted because of collinearity
note: _Iauthority_1097 omitted because of collinearity
note: _Iauthority_1098 omitted because of collinearity
note: _Iauthority_1099 omitted because of collinearity
note: _Iauthority_1100 omitted because of collinearity
note: _Iauthority_1101 omitted because of collinearity
note: _Iauthority_1102 omitted because of collinearity
note: _Iauthority_1103 omitted because of collinearity
note: _Iauthority_1104 omitted because of collinearity
note: _Iauthority_1105 omitted because of collinearity
note: _Iauthority_1106 omitted because of collinearity
note: _Iauthority_1107 omitted because of collinearity
note: _Iauthority_1108 omitted because of collinearity
note: _Iauthority_1109 omitted because of collinearity
note: _Iauthority_1110 omitted because of collinearity
note: _Iauthority_1111 omitted because of collinearity
note: _Iauthority_1112 omitted because of collinearity
note: _Iauthority_1113 omitted because of collinearity
note: _Iauthority_1114 omitted because of collinearity
note: _Iauthority_1115 omitted because of collinearity
note: _Iauthority_1116 omitted because of collinearity
note: _Iauthority_1117 omitted because of collinearity
note: _Iauthority_1118 omitted because of collinearity
note: _Iauthority_1119 omitted because of collinearity
note: _Iauthority_1120 omitted because of collinearity
note: _Iauthority_1121 omitted because of collinearity
note: _Iauthority_1122 omitted because of collinearity
note: _Iauthority_1123 omitted because of collinearity
note: _Iauthority_1124 omitted because of collinearity
note: _Iauthority_1125 omitted because of collinearity
note: _Iauthority_1126 omitted because of collinearity
note: _Iauthority_1127 omitted because of collinearity
note: _Iauthority_1128 omitted because of collinearity
note: _Iauthority_1129 omitted because of collinearity
note: _Iauthority_1130 omitted because of collinearity
note: _Iauthority_1131 omitted because of collinearity
note: _Iauthority_1132 omitted because of collinearity
note: _Iauthority_1133 omitted because of collinearity
note: _Iauthority_1134 omitted because of collinearity
note: _Iauthority_1135 omitted because of collinearity
note: _Iauthority_1136 omitted because of collinearity
note: _Iauthority_1137 omitted because of collinearity
note: _Iauthority_1138 omitted because of collinearity
note: _Iauthority_1139 omitted because of collinearity
note: _Iauthority_1140 omitted because of collinearity
note: _Iauthority_1141 omitted because of collinearity
note: _Iauthority_1142 omitted because of collinearity
note: _Iauthority_1143 omitted because of collinearity
note: _Iauthority_1144 omitted because of collinearity
note: _Iauthority_1145 omitted because of collinearity
note: _Iauthority_1146 omitted because of collinearity
note: _Iauthority_1147 omitted because of collinearity
note: _Iauthority_1148 omitted because of collinearity
note: _Iauthority_1149 omitted because of collinearity
note: _Iauthority_1150 omitted because of collinearity
note: _Iauthority_1151 omitted because of collinearity
note: _Iauthority_1152 omitted because of collinearity
note: _Iauthority_1153 omitted because of collinearity
note: _Iauthority_1154 omitted because of collinearity
note: _Iauthority_1155 omitted because of collinearity
note: _Iauthority_1156 omitted because of collinearity
note: _Iauthority_1157 omitted because of collinearity
note: _Iauthority_1158 omitted because of collinearity
note: _Iauthority_1159 omitted because of collinearity
note: _Iauthority_1160 omitted because of collinearity
note: _Iauthority_1161 omitted because of collinearity
note: _Iauthority_1162 omitted because of collinearity
note: _Iauthority_1163 omitted because of collinearity
note: _Iauthority_1164 omitted because of collinearity
note: _Iauthority_1165 omitted because of collinearity
note: _Iauthority_1166 omitted because of collinearity
note: _Iauthority_1167 omitted because of collinearity
note: _Iauthority_1168 omitted because of collinearity
note: _Iauthority_1169 omitted because of collinearity
note: _Iauthority_1170 omitted because of collinearity
note: _Iauthority_1171 omitted because of collinearity
note: _Iauthority_1172 omitted because of collinearity
note: _Iauthority_1173 omitted because of collinearity
note: _Iauthority_1174 omitted because of collinearity
note: _Iauthority_1175 omitted because of collinearity
note: _Iauthority_1176 omitted because of collinearity
note: _Iauthority_1177 omitted because of collinearity
note: _Iauthority_1178 omitted because of collinearity
note: _Iauthority_1179 omitted because of collinearity
note: _Iauthority_1180 omitted because of collinearity
note: _Iauthority_1181 omitted because of collinearity
note: _Iauthority_1182 omitted because of collinearity
note: _Iauthority_1183 omitted because of collinearity
note: _Iauthority_1184 omitted because of collinearity
note: _Iauthority_1185 omitted because of collinearity
note: _Iauthority_1186 omitted because of collinearity
note: _Iauthority_1187 omitted because of collinearity
note: _Iauthority_1188 omitted because of collinearity
note: _Iauthority_1189 omitted because of collinearity
note: _Iauthority_1190 omitted because of collinearity
note: _Iauthority_1191 omitted because of collinearity
note: _Iauthority_1192 omitted because of collinearity
note: _Iauthority_1193 omitted because of collinearity
note: _Iauthority_1194 omitted because of collinearity
note: _Iauthority_1195 omitted because of collinearity
note: _Iauthority_1196 omitted because of collinearity
note: _Iauthority_1197 omitted because of collinearity
note: _Iauthority_1198 omitted because of collinearity
note: _Iauthority_1199 omitted because of collinearity
note: _Iauthority_1200 omitted because of collinearity
note: _Iauthority_1201 omitted because of collinearity
note: _Iauthority_1202 omitted because of collinearity
note: _Iauthority_1203 omitted because of collinearity
note: _Iauthority_1204 omitted because of collinearity
note: _Iauthority_1205 omitted because of collinearity
note: _Iauthority_1206 omitted because of collinearity
note: _Iauthority_1207 omitted because of collinearity
note: _Iauthority_1208 omitted because of collinearity
note: _Iauthority_1209 omitted because of collinearity
note: _Iauthority_1210 omitted because of collinearity
note: _Iauthority_1211 omitted because of collinearity
note: _Iauthority_1212 omitted because of collinearity
note: _Iauthority_1213 omitted because of collinearity
note: _Iauthority_1214 omitted because of collinearity
note: _Iauthority_1215 omitted because of collinearity
note: _Iauthority_1216 omitted because of collinearity
note: _Iauthority_1217 omitted because of collinearity
note: _Iauthority_1218 omitted because of collinearity
note: _Iauthority_1219 omitted because of collinearity
note: _Iauthority_1220 omitted because of collinearity
note: _Iauthority_1221 omitted because of collinearity
note: _Iauthority_1222 omitted because of collinearity
note: _Iauthority_1223 omitted because of collinearity
note: _Iauthority_1224 omitted because of collinearity
note: _Iauthority_1225 omitted because of collinearity
note: _Iauthority_1226 omitted because of collinearity
note: _Iauthority_1227 omitted because of collinearity
note: _Iauthority_1228 omitted because of collinearity
note: _Iauthority_1229 omitted because of collinearity
note: _Iauthority_1230 omitted because of collinearity
note: _Iauthority_1231 omitted because of collinearity
note: _Iauthority_1232 omitted because of collinearity
note: _Iauthority_1233 omitted because of collinearity
note: _Iauthority_1234 omitted because of collinearity
note: _Iauthority_1235 omitted because of collinearity
note: _Iauthority_1236 omitted because of collinearity
note: _Iauthority_1237 omitted because of collinearity
note: _Iauthority_1238 omitted because of collinearity
note: _Iauthority_1239 omitted because of collinearity
note: _Iauthority_1240 omitted because of collinearity
note: _Iauthority_1241 omitted because of collinearity
note: _Iauthority_1242 omitted because of collinearity
note: _Iauthority_1243 omitted because of collinearity
note: _Iauthority_1244 omitted because of collinearity
note: _Iauthority_1245 omitted because of collinearity
note: _Iauthority_1247 omitted because of collinearity
note: _Iauthority_1248 omitted because of collinearity
note: _Iauthority_1249 omitted because of collinearity
note: _Iauthority_1250 omitted because of collinearity
note: _Iauthority_1251 omitted because of collinearity
note: _Iauthority_1252 omitted because of collinearity
note: _Iauthority_1253 omitted because of collinearity
note: _Iauthority_1254 omitted because of collinearity
note: _Iauthority_1255 omitted because of collinearity
note: _Iauthority_1256 omitted because of collinearity
note: _Iauthority_1257 omitted because of collinearity
note: _Iauthority_1258 omitted because of collinearity
note: _Iauthority_1259 omitted because of collinearity
note: _Iauthority_1260 omitted because of collinearity
note: _Iauthority_1261 omitted because of collinearity
note: _Iauthority_1262 omitted because of collinearity
note: _Iauthority_1263 omitted because of collinearity
note: _Iauthority_1264 omitted because of collinearity
note: _Iauthority_1265 omitted because of collinearity
note: _Iauthority_1266 omitted because of collinearity
note: _Iauthority_1267 omitted because of collinearity
note: _Iauthority_1268 omitted because of collinearity
note: _Iauthority_1269 omitted because of collinearity
note: _Iauthority_1270 omitted because of collinearity
note: _Iauthority_1271 omitted because of collinearity
note: _Iauthority_1272 omitted because of collinearity
note: _Iauthority_1273 omitted because of collinearity
note: _Iauthority_1274 omitted because of collinearity
note: _Iauthority_1275 omitted because of collinearity
note: _Iauthority_1276 omitted because of collinearity
note: _Iauthority_1277 omitted because of collinearity
note: _Iauthority_1278 omitted because of collinearity
note: _Iauthority_1279 omitted because of collinearity
note: _Iauthority_1280 omitted because of collinearity
note: _Iauthority_1281 omitted because of collinearity
note: _Iauthority_1282 omitted because of collinearity
note: _Iauthority_1283 omitted because of collinearity
note: _Iauthority_1284 omitted because of collinearity
note: _Iauthority_1285 omitted because of collinearity
note: _Iauthority_1286 omitted because of collinearity
note: _Iauthority_1287 omitted because of collinearity
note: _Iauthority_1288 omitted because of collinearity
note: _Iauthority_1289 omitted because of collinearity
note: _Iauthority_1290 omitted because of collinearity
note: _Iauthority_1291 omitted because of collinearity
note: _Iauthority_1292 omitted because of collinearity
note: _Iauthority_1293 omitted because of collinearity
note: _Iauthority_1294 omitted because of collinearity
note: _Iauthority_1295 omitted because of collinearity
note: _Iauthority_1296 omitted because of collinearity
note: _Iauthority_1297 omitted because of collinearity
note: _Iauthority_1298 omitted because of collinearity
note: _Iauthority_1299 omitted because of collinearity
note: _Iauthority_1300 omitted because of collinearity
note: _Iauthority_1301 omitted because of collinearity
note: _Iauthority_1302 omitted because of collinearity
note: _Iauthority_1303 omitted because of collinearity
note: _Iauthority_1304 omitted because of collinearity
note: _Iauthority_1305 omitted because of collinearity
note: _Iauthority_1306 omitted because of collinearity
note: _Iauthority_1307 omitted because of collinearity
note: _Iauthority_1308 omitted because of collinearity
note: _Iauthority_1309 omitted because of collinearity
note: _Iauthority_1310 omitted because of collinearity
note: _Iauthority_1311 omitted because of collinearity
note: _Iauthority_1312 omitted because of collinearity
note: _Iauthority_1313 omitted because of collinearity
note: _Iauthority_1314 omitted because of collinearity
note: _Iauthority_1315 omitted because of collinearity
note: _Iauthority_1316 omitted because of collinearity
note: _Iauthority_1317 omitted because of collinearity
note: _Iauthority_1318 omitted because of collinearity
note: _Iauthority_1319 omitted because of collinearity
note: _Iauthority_1320 omitted because of collinearity
note: _Iauthority_1321 omitted because of collinearity
note: _Iauthority_1322 omitted because of collinearity
note: _Iauthority_1323 omitted because of collinearity
note: _Iauthority_1324 omitted because of collinearity
note: _Iauthority_1325 omitted because of collinearity
note: _Iauthority_1326 omitted because of collinearity
note: _Iauthority_1327 omitted because of collinearity
note: _Iauthority_1328 omitted because of collinearity
note: _Iauthority_1329 omitted because of collinearity
note: _Iauthority_1330 omitted because of collinearity
note: _Iauthority_1331 omitted because of collinearity
note: _Iauthority_1332 omitted because of collinearity
note: _Iauthority_1333 omitted because of collinearity
note: _Iauthority_1334 omitted because of collinearity
note: _Iauthority_1335 omitted because of collinearity
note: _Iauthority_1336 omitted because of collinearity
note: _Iauthority_1337 omitted because of collinearity
note: _Iauthority_1338 omitted because of collinearity
note: _Iauthority_1339 omitted because of collinearity
note: _Iauthority_1340 omitted because of collinearity
note: _Iauthority_1341 omitted because of collinearity
note: _Iauthority_1342 omitted because of collinearity
note: _Iauthority_1343 omitted because of collinearity
note: _Iauthority_1344 omitted because of collinearity
note: _Iauthority_1345 omitted because of collinearity
note: _Iauthority_1346 omitted because of collinearity
note: _Iauthority_1347 omitted because of collinearity
note: _Iauthority_1348 omitted because of collinearity
note: _Iauthority_1349 omitted because of collinearity
note: _Iauthority_1350 omitted because of collinearity
note: _Iauthority_1351 omitted because of collinearity
note: _Iauthority_1352 omitted because of collinearity
note: _Iauthority_1354 omitted because of collinearity
note: _Iauthority_1355 omitted because of collinearity
note: _Iauthority_1356 omitted because of collinearity
note: _Iauthority_1357 omitted because of collinearity
note: _Iauthority_1358 omitted because of collinearity
note: _Iauthority_1359 omitted because of collinearity
note: _Iauthority_1360 omitted because of collinearity
note: _Iauthority_1361 omitted because of collinearity
note: _Iauthority_1362 omitted because of collinearity
note: _Iauthority_1363 omitted because of collinearity
note: _Iauthority_1364 omitted because of collinearity
note: _Iauthority_1365 omitted because of collinearity
note: _Iauthority_1366 omitted because of collinearity
note: _Iauthority_1367 omitted because of collinearity
note: _Iauthority_1368 omitted because of collinearity
note: _Iauthority_1369 omitted because of collinearity
note: _Iauthority_1370 omitted because of collinearity
note: _Iauthority_1371 omitted because of collinearity
note: _Iauthority_1372 omitted because of collinearity
note: _Iauthority_1373 omitted because of collinearity
note: _Iauthority_1374 omitted because of collinearity
note: _Iauthority_1375 omitted because of collinearity
note: _Iauthority_1376 omitted because of collinearity
note: _Iauthority_1377 omitted because of collinearity
note: _Iauthority_1378 omitted because of collinearity
note: _Iauthority_1379 omitted because of collinearity
note: _Iauthority_1380 omitted because of collinearity
note: _Iauthority_1381 omitted because of collinearity
note: _Iauthority_1382 omitted because of collinearity
note: _Iauthority_1383 omitted because of collinearity
note: _Iauthority_1384 omitted because of collinearity
note: _Iauthority_1385 omitted because of collinearity
note: _Iauthority_1386 omitted because of collinearity
note: _Iauthority_1387 omitted because of collinearity
note: _Iauthority_1388 omitted because of collinearity
note: _Iauthority_1389 omitted because of collinearity
note: _Iauthority_1390 omitted because of collinearity
note: _Iauthority_1391 omitted because of collinearity
note: _Iauthority_1392 omitted because of collinearity
note: _Iauthority_1393 omitted because of collinearity
note: _Iauthority_1394 omitted because of collinearity
note: _Iauthority_1395 omitted because of collinearity
note: _Iauthority_1396 omitted because of collinearity
note: _Iauthority_1397 omitted because of collinearity
note: _Iauthority_1398 omitted because of collinearity
note: _Iauthority_1399 omitted because of collinearity
note: _Iauthority_1400 omitted because of collinearity
note: _Iauthority_1401 omitted because of collinearity
note: _Iauthority_1402 omitted because of collinearity
note: _Iauthority_1403 omitted because of collinearity
note: _Iauthority_1404 omitted because of collinearity
note: _Iauthority_1405 omitted because of collinearity
note: _Iauthority_1406 omitted because of collinearity
note: _Iauthority_1407 omitted because of collinearity
note: _Iauthority_1408 omitted because of collinearity
note: _Iauthority_1409 omitted because of collinearity
note: _Iauthority_1410 omitted because of collinearity
note: _Iauthority_1411 omitted because of collinearity
note: _Iauthority_1412 omitted because of collinearity
note: _Iauthority_1413 omitted because of collinearity
note: _Iauthority_1414 omitted because of collinearity
note: _Iauthority_1415 omitted because of collinearity
note: _Iauthority_1416 omitted because of collinearity
note: _Iauthority_1417 omitted because of collinearity
note: _Iauthority_1418 omitted because of collinearity
note: _Iauthority_1419 omitted because of collinearity
note: _Iauthority_1420 omitted because of collinearity
note: _Iauthority_1421 omitted because of collinearity
note: _Iauthority_1422 omitted because of collinearity
note: _Iauthority_1423 omitted because of collinearity
note: _Iauthority_1424 omitted because of collinearity
note: _Iauthority_1425 omitted because of collinearity
note: _Iauthority_1426 omitted because of collinearity
note: _Iauthority_1427 omitted because of collinearity
note: _Iauthority_1428 omitted because of collinearity
note: _Iauthority_1429 omitted because of collinearity
note: _Iauthority_1430 omitted because of collinearity
note: _Iauthority_1431 omitted because of collinearity
note: _Iauthority_1432 omitted because of collinearity
note: _Iauthority_1433 omitted because of collinearity
note: _Iauthority_1434 omitted because of collinearity
note: _Iauthority_1435 omitted because of collinearity
note: _Iauthority_1436 omitted because of collinearity
note: _Iauthority_1437 omitted because of collinearity
note: _Iauthority_1438 omitted because of collinearity
note: _Iauthority_1439 omitted because of collinearity
note: _Iauthority_1440 omitted because of collinearity
note: _Iauthority_1441 omitted because of collinearity
note: _Iauthority_1442 omitted because of collinearity
note: _Iauthority_1443 omitted because of collinearity
note: _Iauthority_1444 omitted because of collinearity
note: _Iauthority_1445 omitted because of collinearity
note: _Iauthority_1446 omitted because of collinearity
note: _Iauthority_1447 omitted because of collinearity
note: _Iauthority_1448 omitted because of collinearity
note: _Iauthority_1449 omitted because of collinearity
note: _Iauthority_1450 omitted because of collinearity
note: _Iauthority_1451 omitted because of collinearity
note: _Iauthority_1452 omitted because of collinearity
note: _Iauthority_1453 omitted because of collinearity
note: _Iauthority_1454 omitted because of collinearity
note: _Iauthority_1455 omitted because of collinearity
note: _Iauthority_1456 omitted because of collinearity
note: _Iauthority_1457 omitted because of collinearity
note: _Iauthority_1458 omitted because of collinearity
note: _Iauthority_1459 omitted because of collinearity
note: _Iauthority_1460 omitted because of collinearity
note: _Iauthority_1461 omitted because of collinearity
note: _Iauthority_1462 omitted because of collinearity
note: _Iauthority_1463 omitted because of collinearity
note: _Iauthority_1464 omitted because of collinearity
note: _Iauthority_1465 omitted because of collinearity
note: _Iauthority_1466 omitted because of collinearity
note: _Iauthority_1467 omitted because of collinearity
note: _Iauthority_1468 omitted because of collinearity
note: _Iauthority_1469 omitted because of collinearity
note: _Iauthority_1470 omitted because of collinearity
note: _Iauthority_1471 omitted because of collinearity
note: _Iauthority_1472 omitted because of collinearity
note: _Iauthority_1473 omitted because of collinearity
note: _Iauthority_1474 omitted because of collinearity
note: _Iauthority_1475 omitted because of collinearity
note: _Iauthority_1476 omitted because of collinearity
note: _Iauthority_1477 omitted because of collinearity
note: _Iauthority_1478 omitted because of collinearity
note: _Iauthority_1479 omitted because of collinearity
note: _Iauthority_1480 omitted because of collinearity
note: _Iauthority_1481 omitted because of collinearity
note: _Iauthority_1482 omitted because of collinearity
note: _Iauthority_1483 omitted because of collinearity
note: _Iauthority_1484 omitted because of collinearity
note: _Iauthority_1485 omitted because of collinearity
note: _Iauthority_1486 omitted because of collinearity
note: _Iauthority_1487 omitted because of collinearity
note: _Iauthority_1488 omitted because of collinearity
note: _Iauthority_1489 omitted because of collinearity
note: _Iauthority_1490 omitted because of collinearity
note: _Iauthority_1491 omitted because of collinearity
note: _Iauthority_1492 omitted because of collinearity
note: _Iauthority_1493 omitted because of collinearity
note: _Iauthority_1494 omitted because of collinearity
note: _Iauthority_1495 omitted because of collinearity
note: _Iauthority_1497 omitted because of collinearity
note: _Iauthority_1498 omitted because of collinearity
note: _Iauthority_1499 omitted because of collinearity
note: _Iauthority_1500 omitted because of collinearity
note: _Iauthority_1501 omitted because of collinearity
note: _Iauthority_1502 omitted because of collinearity
note: _Iauthority_1503 omitted because of collinearity
note: _Iauthority_1504 omitted because of collinearity
note: _Iauthority_1505 omitted because of collinearity
note: _Iauthority_1506 omitted because of collinearity
note: _Iauthority_1507 omitted because of collinearity
note: _Iauthority_1508 omitted because of collinearity
note: _Iauthority_1509 omitted because of collinearity
note: _Iauthority_1510 omitted because of collinearity
note: _Iauthority_1511 omitted because of collinearity
note: _Iauthority_1512 omitted because of collinearity
note: _Iauthority_1513 omitted because of collinearity
note: _Iauthority_1514 omitted because of collinearity
note: _Iauthority_1515 omitted because of collinearity
note: _Iauthority_1516 omitted because of collinearity
note: _Iauthority_1517 omitted because of collinearity
note: _Iauthority_1518 omitted because of collinearity
note: _Iauthority_1519 omitted because of collinearity
note: _Iauthority_1520 omitted because of collinearity
note: _Iauthority_1521 omitted because of collinearity
note: _Iauthority_1522 omitted because of collinearity
note: _Iauthority_1523 omitted because of collinearity
note: _Iauthority_1524 omitted because of collinearity
note: _Iauthority_1525 omitted because of collinearity
note: _Iauthority_1526 omitted because of collinearity
note: _Iauthority_1527 omitted because of collinearity
note: _Iauthority_1528 omitted because of collinearity
note: _Iauthority_1529 omitted because of collinearity
note: _Iauthority_1530 omitted because of collinearity
note: _Iauthority_1531 omitted because of collinearity
note: _Iauthority_1532 omitted because of collinearity
note: _Iauthority_1533 omitted because of collinearity
note: _Iauthority_1534 omitted because of collinearity
note: _Iauthority_1535 omitted because of collinearity
note: _Iauthority_1536 omitted because of collinearity
note: _Iauthority_1537 omitted because of collinearity
note: _Iauthority_1538 omitted because of collinearity
note: _Iauthority_1539 omitted because of collinearity
note: _Iauthority_1540 omitted because of collinearity
note: _Iauthority_1541 omitted because of collinearity
note: _Iauthority_1542 omitted because of collinearity
note: _Iauthority_1543 omitted because of collinearity
note: _Iauthority_1544 omitted because of collinearity
note: _Iauthority_1545 omitted because of collinearity
note: _Iauthority_1546 omitted because of collinearity
note: _Iauthority_1547 omitted because of collinearity
note: _Iauthority_1548 omitted because of collinearity
note: _Iauthority_1549 omitted because of collinearity
note: _Iauthority_1550 omitted because of collinearity
note: _Iauthority_1551 omitted because of collinearity
note: _Iauthority_1552 omitted because of collinearity
note: _Iauthority_1553 omitted because of collinearity
note: _Iauthority_1554 omitted because of collinearity
note: _Iauthority_1555 omitted because of collinearity
note: _Iauthority_1556 omitted because of collinearity
note: _Iauthority_1557 omitted because of collinearity
note: _Iauthority_1558 omitted because of collinearity
note: _Iauthority_1559 omitted because of collinearity
note: _Iauthority_1560 omitted because of collinearity
note: _Iauthority_1561 omitted because of collinearity
note: _Iauthority_1562 omitted because of collinearity
note: _Iauthority_1563 omitted because of collinearity
note: _Iauthority_1564 omitted because of collinearity
note: _Iauthority_1565 omitted because of collinearity
note: _Iauthority_1566 omitted because of collinearity
note: _Iauthority_1567 omitted because of collinearity
note: _Iauthority_1568 omitted because of collinearity
note: _Iauthority_1569 omitted because of collinearity
note: _Iauthority_1570 omitted because of collinearity
note: _Iauthority_1571 omitted because of collinearity
note: _Iauthority_1572 omitted because of collinearity
note: _Iauthority_1573 omitted because of collinearity
note: _Iauthority_1574 omitted because of collinearity
note: _Iauthority_1575 omitted because of collinearity
note: _Iauthority_1576 omitted because of collinearity
note: _Iauthority_1577 omitted because of collinearity
note: _Iauthority_1578 omitted because of collinearity
note: _Iauthority_1579 omitted because of collinearity
note: _Iauthority_1580 omitted because of collinearity
note: _Iauthority_1581 omitted because of collinearity
note: _Iauthority_1582 omitted because of collinearity
note: _Iauthority_1583 omitted because of collinearity
note: _Iauthority_1584 omitted because of collinearity
note: _Iauthority_1585 omitted because of collinearity
note: _Iauthority_1586 omitted because of collinearity
note: _Iauthority_1587 omitted because of collinearity
note: _Iauthority_1588 omitted because of collinearity
note: _Iauthority_1589 omitted because of collinearity
note: _Iauthority_1590 omitted because of collinearity
note: _Iauthority_1591 omitted because of collinearity
note: _Iauthority_1592 omitted because of collinearity
note: _Iauthority_1593 omitted because of collinearity
note: _Iauthority_1594 omitted because of collinearity
note: _Iauthority_1595 omitted because of collinearity
note: _Iauthority_1596 omitted because of collinearity
note: _Iauthority_1597 omitted because of collinearity
note: _Iauthority_1598 omitted because of collinearity
note: _Iauthority_1599 omitted because of collinearity
note: _Iauthority_1600 omitted because of collinearity
note: _Iauthority_1601 omitted because of collinearity
note: _Iauthority_1602 omitted because of collinearity
note: _Iauthority_1603 omitted because of collinearity
note: _Iauthority_1604 omitted because of collinearity
note: _Iauthority_1605 omitted because of collinearity
note: _Iauthority_1606 omitted because of collinearity
note: _Iauthority_1607 omitted because of collinearity
note: _Iauthority_1608 omitted because of collinearity
note: _Iauthority_1609 omitted because of collinearity
note: _Iauthority_1610 omitted because of collinearity
note: _Iauthority_1611 omitted because of collinearity
note: _Iauthority_1612 omitted because of collinearity
note: _Iauthority_1613 omitted because of collinearity
note: _Iauthority_1614 omitted because of collinearity
note: _Iauthority_1615 omitted because of collinearity
note: _Iauthority_1616 omitted because of collinearity
note: _Iauthority_1617 omitted because of collinearity
note: _Iauthority_1618 omitted because of collinearity
note: _Iauthority_1619 omitted because of collinearity
note: _Iauthority_1620 omitted because of collinearity
note: _Iauthority_1621 omitted because of collinearity
note: _Iauthority_1622 omitted because of collinearity
note: _Iauthority_1623 omitted because of collinearity
note: _Iauthority_1624 omitted because of collinearity
note: _Iauthority_1625 omitted because of collinearity
note: _Iauthority_1626 omitted because of collinearity
note: _Iauthority_1627 omitted because of collinearity
note: _Iauthority_1628 omitted because of collinearity
note: _Iauthority_1630 omitted because of collinearity
note: _Iauthority_1631 omitted because of collinearity
note: _Iauthority_1632 omitted because of collinearity
note: _Iauthority_1633 omitted because of collinearity
note: _Iauthority_1634 omitted because of collinearity
note: _Iauthority_1635 omitted because of collinearity
note: _Iauthority_1636 omitted because of collinearity
note: _Iauthority_1637 omitted because of collinearity
note: _Iauthority_1638 omitted because of collinearity
note: _Iauthority_1639 omitted because of collinearity
note: _Iauthority_1640 omitted because of collinearity
note: _Iauthority_1641 omitted because of collinearity
note: _Iauthority_1642 omitted because of collinearity
note: _Iauthority_1643 omitted because of collinearity
note: _Iauthority_1644 omitted because of collinearity
note: _Iauthority_1645 omitted because of collinearity
note: _Iauthority_1646 omitted because of collinearity
note: _Iauthority_1647 omitted because of collinearity
note: _Iauthority_1648 omitted because of collinearity
note: _Iauthority_1649 omitted because of collinearity
note: _Iauthority_1650 omitted because of collinearity
note: _Iauthority_1651 omitted because of collinearity
note: _Iauthority_1652 omitted because of collinearity
note: _Iauthority_1653 omitted because of collinearity
note: _Iauthority_1654 omitted because of collinearity
note: _Iauthority_1655 omitted because of collinearity
note: _Iauthority_1656 omitted because of collinearity
note: _Iauthority_1657 omitted because of collinearity
note: _Iauthority_1658 omitted because of collinearity
note: _Iauthority_1659 omitted because of collinearity
note: _Iauthority_1660 omitted because of collinearity
note: _Iauthority_1661 omitted because of collinearity
note: _Iauthority_1662 omitted because of collinearity
note: _Iauthority_1663 omitted because of collinearity
note: _Iauthority_1664 omitted because of collinearity
note: _Iauthority_1665 omitted because of collinearity
note: _Iauthority_1667 omitted because of collinearity
note: _Iauthority_1668 omitted because of collinearity
note: _Iauthority_1669 omitted because of collinearity
note: _Iauthority_1670 omitted because of collinearity
note: _Iauthority_1671 omitted because of collinearity
note: _Iauthority_1672 omitted because of collinearity
note: _Iauthority_1673 omitted because of collinearity
note: _Iauthority_1674 omitted because of collinearity
note: _Iauthority_1675 omitted because of collinearity
note: _Iauthority_1676 omitted because of collinearity
note: _Iauthority_1677 omitted because of collinearity
note: _Iauthority_1678 omitted because of collinearity
note: _Iauthority_1679 omitted because of collinearity
note: _Iauthority_1680 omitted because of collinearity
note: _Iauthority_1681 omitted because of collinearity
note: _Iauthority_1682 omitted because of collinearity
note: _Iauthority_1683 omitted because of collinearity
note: _Iauthority_1684 omitted because of collinearity
note: _Iauthority_1685 omitted because of collinearity
note: _Iauthority_1686 omitted because of collinearity
note: _Iauthority_1687 omitted because of collinearity
note: _Iauthority_1688 omitted because of collinearity
note: _Iauthority_1689 omitted because of collinearity
note: _Iauthority_1690 omitted because of collinearity
note: _Iauthority_1691 omitted because of collinearity
note: _Iauthority_1692 omitted because of collinearity
note: _Iauthority_1693 omitted because of collinearity
note: _Iauthority_1694 omitted because of collinearity
note: _Iauthority_1695 omitted because of collinearity
note: _Iauthority_1696 omitted because of collinearity
note: _Iauthority_1697 omitted because of collinearity
note: _Iauthority_1698 omitted because of collinearity
note: _Iauthority_1699 omitted because of collinearity
note: _Iauthority_1700 omitted because of collinearity
note: _Iauthority_1701 omitted because of collinearity
note: _Iauthority_1702 omitted because of collinearity
note: _Iauthority_1703 omitted because of collinearity
note: _Iauthority_1704 omitted because of collinearity
note: _Iauthority_1705 omitted because of collinearity
note: _Iauthority_1706 omitted because of collinearity
note: _Iauthority_1707 omitted because of collinearity
note: _Iauthority_1708 omitted because of collinearity
note: _Iauthority_1709 omitted because of collinearity
note: _Iauthority_1710 omitted because of collinearity
note: _Iauthority_1711 omitted because of collinearity
note: _Iauthority_1712 omitted because of collinearity
note: _Iauthority_1713 omitted because of collinearity
note: _Iauthority_1714 omitted because of collinearity
note: _Iauthority_1715 omitted because of collinearity
note: _Iauthority_1716 omitted because of collinearity
note: _Iauthority_1717 omitted because of collinearity
note: _Iauthority_1718 omitted because of collinearity
note: _Iauthority_1719 omitted because of collinearity
note: _Iauthority_1720 omitted because of collinearity
note: _Iauthority_1721 omitted because of collinearity
note: _Iauthority_1722 omitted because of collinearity
note: _Iauthority_1723 omitted because of collinearity
note: _Iauthority_1724 omitted because of collinearity
note: _Iauthority_1725 omitted because of collinearity
note: _Iauthority_1726 omitted because of collinearity
note: _Iauthority_1727 omitted because of collinearity
note: _Iauthority_1728 omitted because of collinearity
note: _Iauthority_1729 omitted because of collinearity
note: _Iauthority_1730 omitted because of collinearity
note: _Iauthority_1731 omitted because of collinearity
note: _Iauthority_1732 omitted because of collinearity
note: _Iauthority_1733 omitted because of collinearity
note: _Iauthority_1734 omitted because of collinearity
note: _Iauthority_1735 omitted because of collinearity
note: _Iauthority_1736 omitted because of collinearity
note: _Iauthority_1737 omitted because of collinearity
note: _Iauthority_1738 omitted because of collinearity
note: _Iauthority_1739 omitted because of collinearity
note: _Iauthority_1740 omitted because of collinearity
note: _Iauthority_1741 omitted because of collinearity
note: _Iauthority_1742 omitted because of collinearity
note: _Iauthority_1743 omitted because of collinearity
note: _Iauthority_1744 omitted because of collinearity
note: _Iauthority_1745 omitted because of collinearity
note: _Iauthority_1746 omitted because of collinearity
note: _Iauthority_1747 omitted because of collinearity
note: _Iauthority_1748 omitted because of collinearity
note: _Iauthority_1749 omitted because of collinearity
note: _Iauthority_1750 omitted because of collinearity
note: _Iauthority_1751 omitted because of collinearity
note: _Iauthority_1752 omitted because of collinearity
note: _Iauthority_1753 omitted because of collinearity
note: _Iauthority_1754 omitted because of collinearity
note: _Iauthority_1755 omitted because of collinearity
note: _Iauthority_1756 omitted because of collinearity
note: _Iauthority_1757 omitted because of collinearity
note: _Iauthority_1759 omitted because of collinearity
note: _Iauthority_1760 omitted because of collinearity
note: _Iauthority_1761 omitted because of collinearity
note: _Iauthority_1762 omitted because of collinearity
note: _Iauthority_1763 omitted because of collinearity
note: _Iauthority_1764 omitted because of collinearity
note: _Iauthority_1765 omitted because of collinearity
note: _Iauthority_1766 omitted because of collinearity
note: _Iauthority_1767 omitted because of collinearity
note: _Iauthority_1769 omitted because of collinearity
note: _Iauthority_1770 omitted because of collinearity
note: _Iauthority_1771 omitted because of collinearity
note: _Iauthority_1772 omitted because of collinearity
note: _Iauthority_1773 omitted because of collinearity
note: _Iauthority_1774 omitted because of collinearity
note: _Iauthority_1775 omitted because of collinearity
note: _Iauthority_1776 omitted because of collinearity
note: _Iauthority_1777 omitted because of collinearity
note: _Iauthority_1778 omitted because of collinearity
note: _Iauthority_1779 omitted because of collinearity
note: _Iauthority_1780 omitted because of collinearity
note: _Iauthority_1781 omitted because of collinearity
note: _Iauthority_1782 omitted because of collinearity
note: _Iauthority_1783 omitted because of collinearity
note: _Iauthority_1784 omitted because of collinearity
note: _Iauthority_1785 omitted because of collinearity
note: _Iauthority_1786 omitted because of collinearity
note: _Iauthority_1787 omitted because of collinearity
note: _Iauthority_1788 omitted because of collinearity
note: _Iauthority_1789 omitted because of collinearity
note: _Iauthority_1790 omitted because of collinearity
note: _Iauthority_1791 omitted because of collinearity
note: _Iauthority_1792 omitted because of collinearity
note: _Iauthority_1793 omitted because of collinearity
note: _Iauthority_1794 omitted because of collinearity
note: _Iauthority_1795 omitted because of collinearity
note: _Iauthority_1796 omitted because of collinearity
note: _Iauthority_1797 omitted because of collinearity
note: _Iauthority_1798 omitted because of collinearity
note: _Iauthority_1799 omitted because of collinearity
note: _Iauthority_1800 omitted because of collinearity
note: _Iauthority_1801 omitted because of collinearity
note: _Iauthority_1802 omitted because of collinearity
note: _Iauthority_1803 omitted because of collinearity
note: _Iauthority_1804 omitted because of collinearity
note: _Iauthority_1805 omitted because of collinearity
note: _Iauthority_1806 omitted because of collinearity
note: _Iauthority_1807 omitted because of collinearity
note: _Iauthority_1808 omitted because of collinearity
note: _Iauthority_1809 omitted because of collinearity
note: _Iauthority_1810 omitted because of collinearity
note: _Iauthority_1811 omitted because of collinearity
note: _Iauthority_1812 omitted because of collinearity
note: _Iauthority_1813 omitted because of collinearity
note: _Iauthority_1814 omitted because of collinearity
note: _Iauthority_1815 omitted because of collinearity
note: _Iauthority_1816 omitted because of collinearity
note: _Iauthority_1817 omitted because of collinearity
note: _Iauthority_1818 omitted because of collinearity
note: _Iauthority_1819 omitted because of collinearity
note: _Iauthority_1820 omitted because of collinearity
note: _Iauthority_1821 omitted because of collinearity
note: _Iauthority_1822 omitted because of collinearity
note: _Iauthority_1823 omitted because of collinearity
note: _Iauthority_1824 omitted because of collinearity
note: _Iauthority_1825 omitted because of collinearity
note: _Iauthority_1826 omitted because of collinearity
note: _Iauthority_1827 omitted because of collinearity
note: _Iauthority_1828 omitted because of collinearity
note: _Iauthority_1829 omitted because of collinearity
note: _Iauthority_1830 omitted because of collinearity
note: _Iauthority_1831 omitted because of collinearity
note: _Iauthority_1832 omitted because of collinearity
note: _Iauthority_1833 omitted because of collinearity
note: _Iauthority_1834 omitted because of collinearity
note: _Iauthority_1835 omitted because of collinearity
note: _Iauthority_1836 omitted because of collinearity
note: _Iauthority_1837 omitted because of collinearity
note: _Iauthority_1838 omitted because of collinearity
note: _Iauthority_1839 omitted because of collinearity
note: _Iauthority_1840 omitted because of collinearity
note: _Iauthority_1841 omitted because of collinearity
note: _Iauthority_1842 omitted because of collinearity
note: _Iauthority_1843 omitted because of collinearity
note: _Iauthority_1844 omitted because of collinearity
note: _Iauthority_1845 omitted because of collinearity
note: _Iauthority_1846 omitted because of collinearity
note: _Iauthority_1847 omitted because of collinearity
note: _Iauthority_1848 omitted because of collinearity
note: _Iauthority_1849 omitted because of collinearity
note: _Iauthority_1850 omitted because of collinearity
note: _Iauthority_1851 omitted because of collinearity
note: _Iauthority_1852 omitted because of collinearity
note: _Iauthority_1853 omitted because of collinearity
note: _Iauthority_1854 omitted because of collinearity
note: _Iauthority_1855 omitted because of collinearity
note: _Iauthority_1856 omitted because of collinearity
note: _Iauthority_1857 omitted because of collinearity
note: _Iauthority_1858 omitted because of collinearity
note: _Iauthority_1859 omitted because of collinearity
note: _Iauthority_1860 omitted because of collinearity
note: _Iauthority_1861 omitted because of collinearity
note: _Iauthority_1862 omitted because of collinearity
note: _Iauthority_1863 omitted because of collinearity
note: _Iauthority_1864 omitted because of collinearity
note: _Iauthority_1865 omitted because of collinearity
note: _Iauthority_1866 omitted because of collinearity
note: _Iauthority_1867 omitted because of collinearity
note: _Iauthority_1868 omitted because of collinearity
note: _Iauthority_1869 omitted because of collinearity
note: _Iauthority_1870 omitted because of collinearity
note: _Iauthority_1871 omitted because of collinearity
note: _Iauthority_1872 omitted because of collinearity
note: _Iauthority_1873 omitted because of collinearity
note: _Iauthority_1874 omitted because of collinearity
note: _Iauthority_1875 omitted because of collinearity
note: _Iauthority_1876 omitted because of collinearity
note: _Iauthority_1877 omitted because of collinearity
note: _Iauthority_1878 omitted because of collinearity
note: _Iauthority_1879 omitted because of collinearity
note: _Iauthority_1880 omitted because of collinearity
note: _Iauthority_1881 omitted because of collinearity
note: _Iauthority_1882 omitted because of collinearity
note: _Iauthority_1883 omitted because of collinearity
note: _Iauthority_1884 omitted because of collinearity
note: _Iauthority_1885 omitted because of collinearity
note: _Iauthority_1886 omitted because of collinearity
note: _Iauthority_1887 omitted because of collinearity
note: _Iauthority_1888 omitted because of collinearity
note: _Iauthority_1889 omitted because of collinearity
note: _Iauthority_1890 omitted because of collinearity
note: _Iauthority_1891 omitted because of collinearity
note: _Iauthority_1892 omitted because of collinearity
note: _Iauthority_1893 omitted because of collinearity
note: _Iauthority_1894 omitted because of collinearity
note: _Iauthority_1895 omitted because of collinearity
note: _Iauthority_1896 omitted because of collinearity
note: _Iauthority_1897 omitted because of collinearity
note: _Iauthority_1898 omitted because of collinearity
note: _Iauthority_1899 omitted because of collinearity
note: _Iauthority_1900 omitted because of collinearity
note: _Iauthority_1901 omitted because of collinearity
note: _Iauthority_1902 omitted because of collinearity
note: _Iauthority_1903 omitted because of collinearity
note: _Iauthority_1904 omitted because of collinearity
note: _Iauthority_1905 omitted because of collinearity
note: _Iauthority_1907 omitted because of collinearity
note: _Iauthority_1908 omitted because of collinearity
note: _Iauthority_1909 omitted because of collinearity
note: _Iauthority_1910 omitted because of collinearity
note: _Iauthority_1911 omitted because of collinearity
note: _Iauthority_1912 omitted because of collinearity
note: _Iauthority_1913 omitted because of collinearity
note: _Iauthority_1914 omitted because of collinearity
note: _Iauthority_1915 omitted because of collinearity
note: _Iauthority_1916 omitted because of collinearity
note: _Iauthority_1917 omitted because of collinearity
note: _Iauthority_1918 omitted because of collinearity
note: _Iauthority_1919 omitted because of collinearity
note: _Iauthority_1920 omitted because of collinearity
note: _Iauthority_1921 omitted because of collinearity
note: _Iauthority_1922 omitted because of collinearity
note: _Iauthority_1923 omitted because of collinearity
note: _Iauthority_1924 omitted because of collinearity
note: _Iauthority_1925 omitted because of collinearity
note: _Iauthority_1926 omitted because of collinearity
note: _Iauthority_1927 omitted because of collinearity
note: _Iauthority_1928 omitted because of collinearity
note: _Iauthority_1929 omitted because of collinearity
note: _Iauthority_1930 omitted because of collinearity
note: _Iauthority_1931 omitted because of collinearity
note: _Iauthority_1932 omitted because of collinearity
note: _Iauthority_1933 omitted because of collinearity
note: _Iauthority_1934 omitted because of collinearity
note: _Iauthority_1935 omitted because of collinearity
note: _Iauthority_1936 omitted because of collinearity
note: _Iauthority_1937 omitted because of collinearity
note: _Iauthority_1938 omitted because of collinearity
note: _Iauthority_1939 omitted because of collinearity
note: _Iauthority_1940 omitted because of collinearity
note: _Iauthority_1941 omitted because of collinearity
note: _Iauthority_1942 omitted because of collinearity
note: _Iauthority_1943 omitted because of collinearity
note: _Iauthority_1944 omitted because of collinearity
note: _Iauthority_1945 omitted because of collinearity
note: _Iauthority_1946 omitted because of collinearity
note: _Iauthority_1947 omitted because of collinearity
note: _Iauthority_1948 omitted because of collinearity
note: _Iauthority_1949 omitted because of collinearity
note: _Iauthority_1950 omitted because of collinearity
note: _Iauthority_1951 omitted because of collinearity
note: _Iauthority_1952 omitted because of collinearity
note: _Iauthority_1953 omitted because of collinearity
note: _Iauthority_1954 omitted because of collinearity
note: _Iauthority_1955 omitted because of collinearity
note: _Iauthority_1956 omitted because of collinearity
note: _Iauthority_1957 omitted because of collinearity
note: _Iauthority_1958 omitted because of collinearity
note: _Iauthority_1959 omitted because of collinearity
note: _Iauthority_1960 omitted because of collinearity
note: _Iauthority_1961 omitted because of collinearity
note: _Iauthority_1962 omitted because of collinearity
note: _Iauthority_1963 omitted because of collinearity
note: _Iauthority_1964 omitted because of collinearity
note: _Iauthority_1965 omitted because of collinearity
note: _Iauthority_1966 omitted because of collinearity
note: _Iauthority_1967 omitted because of collinearity
note: _Iauthority_1968 omitted because of collinearity
note: _Iauthority_1969 omitted because of collinearity
note: _Iauthority_1970 omitted because of collinearity
note: _Iauthority_1971 omitted because of collinearity
note: _Iauthority_1972 omitted because of collinearity
note: _Iauthority_1973 omitted because of collinearity
note: _Iauthority_1974 omitted because of collinearity
note: _Iauthority_1975 omitted because of collinearity
note: _Iauthority_1976 omitted because of collinearity
note: _Iauthority_1977 omitted because of collinearity
note: _Iauthority_1979 omitted because of collinearity
note: _Iauthority_1980 omitted because of collinearity
note: _Iauthority_1981 omitted because of collinearity
note: _Iauthority_1982 omitted because of collinearity
note: _Iauthority_1983 omitted because of collinearity
note: _Iauthority_1984 omitted because of collinearity
note: _Iauthority_1985 omitted because of collinearity
note: _Iauthority_1986 omitted because of collinearity
note: _Iauthority_1987 omitted because of collinearity
note: _Iauthority_1988 omitted because of collinearity
note: _Iauthority_1989 omitted because of collinearity
note: _Iauthority_1990 omitted because of collinearity
note: _Iauthority_1991 omitted because of collinearity
note: _Iauthority_1992 omitted because of collinearity
note: _Iauthority_1993 omitted because of collinearity
note: _Iauthority_1994 omitted because of collinearity
note: _Iauthority_1995 omitted because of collinearity
note: _Iauthority_1996 omitted because of collinearity
note: _Iauthority_1997 omitted because of collinearity
note: _Iauthority_1998 omitted because of collinearity
note: _Iauthority_1999 omitted because of collinearity
note: _Iauthority_2000 omitted because of collinearity
note: _Iauthority_2001 omitted because of collinearity
note: _Iauthority_2002 omitted because of collinearity
note: _Iauthority_2003 omitted because of collinearity
note: _Iauthority_2004 omitted because of collinearity
note: _Iauthority_2005 omitted because of collinearity
note: _Iauthority_2006 omitted because of collinearity
note: _Iauthority_2007 omitted because of collinearity
note: _Iauthority_2008 omitted because of collinearity
note: _Iauthority_2009 omitted because of collinearity
note: _Iauthority_2010 omitted because of collinearity
note: _Iauthority_2011 omitted because of collinearity
note: _Iauthority_2012 omitted because of collinearity
note: _Iauthority_2013 omitted because of collinearity
note: _Iauthority_2014 omitted because of collinearity
note: _Iauthority_2016 omitted because of collinearity
note: _Iauthority_2017 omitted because of collinearity
note: _Iauthority_2018 omitted because of collinearity
note: _Iauthority_2019 omitted because of collinearity
note: _Iauthority_3070001 omitted because of collinearity
note: _Iauthority_3090272 omitted because of collinearity
